# ML Applicaiton: Applying ML Models to the transition data

### Tasks included:
- Reading transition matrices 
- Reading P/AUC scores 
- Apply ML models 
- Select features and apply ML models 

## Imports and system info

In [101]:
import HumachLab_Global 
HumachLab_Global.get_system_info()


List of OS platforms and codes
___________________________________________
0 Darwin
1 Windows
2 Linux
===> "1 - Windows" OS is detected.

Processor (CPU) details: 
___________________________________________
{'python_version': '3.7.10.final.0 (64 bit)', 'cpuinfo_version': [8, 0, 0], 'cpuinfo_version_string': '8.0.0', 'arch': 'X86_64', 'bits': 64, 'count': 40, 'arch_string_raw': 'AMD64', 'vendor_id_raw': 'GenuineIntel', 'brand_raw': 'Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz', 'hz_advertised_friendly': '2.2000 GHz', 'hz_actual_friendly': '2.2010 GHz', 'hz_advertised': [2200000000, 0], 'hz_actual': [2201000000, 0], 'l2_cache_size': 2621440, 'stepping': 1, 'model': 79, 'family': 6, 'l3_cache_size': 26214400, 'flags': ['3dnow', '3dnowprefetch', 'abm', 'acpi', 'adx', 'aes', 'apic', 'avx', 'avx2', 'bmi1', 'bmi2', 'clflush', 'cmov', 'cx16', 'cx8', 'dca', 'de', 'ds_cpl', 'dtes64', 'dts', 'erms', 'est', 'f16c', 'fma', 'fpu', 'fxsr', 'hle', 'ht', 'ia64', 'intel_pt', 'invpcid', 'lahf_lm', 'mca',

(1,
 'Windows',
 {'brand_raw': 'Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz',
  'arch_string_raw': 'AMD64',
  'arch': 'X86_64',
  'count': 40,
  'python_version': '3.7.10.final.0 (64 bit)',
  'CPU_usage': 60.5,
  'RAM_usage': 88.7,
  'Total_RAM': 127.9,
  'Used_RAM': 113.4,
  'Available_RAM': 14.5},
 [1, 2, 3, 4, 5, 6, 7],
 1)

In [2]:
'''
Importing necessary modules
'''

import os
import sys
import datetime

print(os.getcwd())
os.chdir(os.getcwd())
print(f"{os.getcwd()}\HumachLab")
sys.path.append(f"{os.getcwd()}\HumachLab")
sys.path.insert(0, os.path.abspath('./HumachLab'))

import itertools as it
import re
import pickle
import json

import copy
from pprint import pprint

import glob

import math
import numbers

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import rc, rcParams
%matplotlib inline
import seaborn as sns


from sklearn.utils import resample
# from imblearn.over_sampling import SMOTE

import HumachLab_Global
from HumachLab import * 
# from HumachLab.HumachLab_Global import *
# import HumachLab_Global
HumachLab_Global.get_system_info()

import mne

# plt.rcParams["figure.figsize"] = plt.rcParamsDefault["figure.figsize"]
plt.rcParams["figure.figsize"] = (20,6)

C:\Users\aliem\OneDrive - Deakin University\_MyResearch\PhD_Research\HML_IHC_Sleep_Data_Analysis
C:\Users\aliem\OneDrive - Deakin University\_MyResearch\PhD_Research\HML_IHC_Sleep_Data_Analysis\HumachLab

List of OS platforms and codes
___________________________________________
0 Darwin
1 Windows
2 Linux
===> "1 - Windows" OS is detected.

Processor (CPU) details: 
___________________________________________
{'python_version': '3.7.10.final.0 (64 bit)', 'cpuinfo_version': [8, 0, 0], 'cpuinfo_version_string': '8.0.0', 'arch': 'X86_64', 'bits': 64, 'count': 40, 'arch_string_raw': 'AMD64', 'vendor_id_raw': 'GenuineIntel', 'brand_raw': 'Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz', 'hz_advertised_friendly': '2.2000 GHz', 'hz_actual_friendly': '2.2010 GHz', 'hz_advertised': [2200000000, 0], 'hz_actual': [2201000000, 0], 'l2_cache_size': 2621440, 'stepping': 1, 'model': 79, 'family': 6, 'l3_cache_size': 26214400, 'flags': ['3dnow', '3dnowprefetch', 'abm', 'acpi', 'adx', 'aes', 'apic', 'avx', 

## Get directory list: Subject-wise

In [3]:
'''
Explore the contents/files in the directory
'''

def get_list_of_paths_from_a_directory(directory, path_type=None, containes=None, extension=None, exclude=None):
    '''
    directory: valid path string, path_type: p_file|p_dir, containes: string, extension: valid string file extension 
    '''
    os_path = os.path
    list_of_paths = []
        
    path_keywords = "*"
    if containes:
        path_keywords = f"{path_keywords}{containes}*"
    
    if extension:
        path_keywords = f"{path_keywords}.{extension}"
        
    complete_path = f"{directory}/{path_keywords}"
    print(f"============> {path_keywords}, {path_type}, {complete_path}")
    
    all_paths = glob.glob(complete_path) 
    all_temp_paths = None
    list_of_paths = None
    
    if path_type:
        if path_type=="p_file":
            all_temp_paths = [path.replace("\\", "/") for path in all_paths if (os_path.exists(path) and os_path.isfile(path))]
        if path_type=="p_dir":
            all_temp_paths = [path.replace("\\", "/") for path in all_paths if (os_path.exists(path) and os_path.isdir(path))]   
    else:
        all_temp_paths = [path.replace("\\", "/") for path in all_paths]
        
    if exclude:
        # print(all_temp_paths)
        # print(len(all_temp_paths), exclude)
        # list_of_paths = [path for path in all_temp_paths for ex in exclude if ex not in path]
        # list_of_paths = [path for ex in exclude for path in all_temp_paths if ex not in path]
        list_of_paths = [path for path in all_temp_paths if not any((ex in path) for ex in exclude)]
        # list_of_paths = [path for ex in exclude if any(ex not in path for path in all_temp_paths)]
        # any(substring in string for substring in substring_list)
        # print(len(list_of_paths))
    else:
        list_of_paths = all_temp_paths.copy()
    
    return list_of_paths

In [4]:
root_directory = "./Results/" 
data_dir = "/_Combined/"
# data_directory = "./Results/" 
# tran_directory = "/Transition_Matrices"
# result_directory = "./Results/_Combined" 
result_subdirctory = "/_Classification"
prob_cal_from_all = False  
result_subdir = ['STP_From_Same_Stages', 'STP_From_All_Stages'][int(prob_cal_from_all)]
data_directory = f"{root_directory}{data_dir}{result_subdir}"
result_directory = f"{root_directory}{result_subdirctory}" 
wake_state_trimmed = True  
result_subdirectory = f"Subject_One_Night{'_TrimW' if wake_state_trimmed else ''}"  ###"Subject_Combined_Record"  "Subject_Separate_Record"  "Subject_One_Night" "Subject_One_Night_TrimW"   ## Change for new type of result 
data_subdirectory = ["CAP_Sleep", "Sleep_EDFX", "SDRC"] 
metadata_subdirectory = ["CAP_Sleep", "Sleep_EDFX", "SDRC"] 
tran_matrix_type = ["count", "dura", "proba"] 
annotation_type = ['annot', 'tran'] 
tran_step = 3
exclude_contents_in_dataset_directory = ["SHA256SUMS", "RECORDS"]
exclude_contents_in_result_directory = ["SHA256SUMS", "RECORDS", "all_annotions", "annot_sequence", "transition_sequence", "hypno", "DATASET_CHANGELOG"]
sleep_stage_labels = ['W', 'S1', 'S2', 'S3', 'S4', 'REM']
sleep_stage_labels_dict = {'W':0, 'S1':1, 'S2':2, 'S3':3, 'S4':4, 'REM':5}
sleep_stage_names_dict = {'W':0, 'S1':1, 'S2':2, 'S3':3, 'S4':4, 'R':5}
list_of_paths = None 
data_directory, result_directory

# directory = dataset_directory
# # list_of_paths = get_list_of_paths_from_a_directory(directory, path_type=None, containes=None, extension=None, exclude=None) 
# # list_of_paths = get_list_of_paths_from_a_directory(directory, path_type="p_file", containes=None, extension=None, exclude=None) 
# # list_of_paths = get_list_of_paths_from_a_directory(directory, path_type="p_dir", containes=None, extension=None, exclude=None) 
# # list_of_paths = get_list_of_paths_from_a_directory(directory, path_type=None, containes="nfle", extension=None, exclude=None) 
# # list_of_paths = get_list_of_paths_from_a_directory(directory, path_type=None, containes=None, extension="edf", exclude=None) 
# list_of_paths = get_list_of_paths_from_a_directory(directory, path_type=None, containes=None, extension=None, exclude=exclude_contents_in_dataset_directory) 
# # list_of_paths = get_list_of_paths_from_a_directory(directory, path_type=None, containes=None, extension=None, exclude=None) 
# # pprint(list_of_paths)
# list_of_paths

('./Results//_Combined/STP_From_Same_Stages', './Results//_Classification')

### Get basic information and transition matrix

In [5]:
def get_demography_info(tmp_dir): 
#     list_of_tran_mat_paths = [] 
    all_demography_df = pd.read_csv(f"{tmp_dir}/all_demography.csv", index_col=False) 
    all_demography_detail_df = pd.read_csv(f"{tmp_dir}/all_demography_detail.csv", index_col=False)  
    return all_demography_df, all_demography_detail_df 


info_type="sub" ##"sub"/"file"  ## Change for new type of result 
annot_type = annotation_type[0]   ## Change for different data preparation for 'annot' and 'tran' 

all_demography_df, all_demography_detail_df = get_demography_info(data_directory) 
all_demography_df  

,#,Dataset,Category_Name,Category,Total_Count,Male_Count,Female_Count,Total_AgeRange,Male_AgeRange,Female_AgeRange
0,1,CAP_Sleep,Bruxism,brux,2,2,0,23 - 34,23 - 34,0 - 0
1,2,CAP_Sleep,Sleep-Disordered Breathing,sdb,4,4,0,65 - 78,65 - 78,0 - 0
2,3,CAP_Sleep,Insomnia,ins,9,4,5,47 - 82,54 - 82,47 - 59
3,4,CAP_Sleep,Narcolepsy,narco,5,2,3,18 - 44,24 - 43,18 - 44
4,5,CAP_Sleep,Nocturnal Frontal Lobe Epilepsy,nfle,40,21,19,14 - 67,14 - 44,16 - 67
5,6,CAP_Sleep,Periodic Leg Movements,plm,10,7,3,40 - 62,40 - 62,50 - 52
6,7,CAP_Sleep,REM Behavior Disorder,rbd,22,19,3,58 - 82,58 - 82,73 - 76
7,8,CAP_Sleep,No Pathology (Controls),n,16,7,9,23 - 42,23 - 34,24 - 42
8,10,CAP_Sleep,Total,NaN,108,66,42,14 - 82,14 - 82,16 - 76
9,11,CAP_Sleep,Sleep Disorders,dis,92,59,33,14 - 82,14 - 82,16 - 76


In [7]:
# '''
# Explore the contents/files in the directory
# '''

# def get_list_of_paths_from_a_directory(directory, path_type=None, containes=None, extension=None, exclude=None):
#     '''
#     directory: valid path string, path_type: p_file|p_dir, containes: string, extension: valid string file extension 
#     '''
#     os_path = os.path
#     list_of_paths = []
        
#     path_keywords = "*"
#     if containes:
#         path_keywords = f"{path_keywords}{containes}*"
    
#     if extension:
#         path_keywords = f"{path_keywords}.{extension}"
        
#     complete_path = f"{directory}/{path_keywords}"
#     print(f"============> {path_keywords}, {path_type}, {complete_path}")
    
#     all_paths = glob.glob(complete_path) 
#     all_temp_paths = None
#     list_of_paths = None
    
#     if path_type:
#         if path_type=="p_file":
#             all_temp_paths = [path.replace("\\", "/") for path in all_paths if (os_path.exists(path) and os_path.isfile(path))]
#         if path_type=="p_dir":
#             all_temp_paths = [path.replace("\\", "/") for path in all_paths if (os_path.exists(path) and os_path.isdir(path))]   
#     else:
#         all_temp_paths = [path.replace("\\", "/") for path in all_paths]
        
#     if exclude:
#         # print(all_temp_paths)
#         # print(len(all_temp_paths), exclude)
#         # list_of_paths = [path for path in all_temp_paths for ex in exclude if ex not in path]
#         # list_of_paths = [path for ex in exclude for path in all_temp_paths if ex not in path]
#         list_of_paths = [path for path in all_temp_paths if not any((ex in path) for ex in exclude)]
#         # list_of_paths = [path for ex in exclude if any(ex not in path for path in all_temp_paths)]
#         # any(substring in string for substring in substring_list)
#         # print(len(list_of_paths))
#     else:
#         list_of_paths = all_temp_paths.copy()
    
#     return list_of_paths

In [8]:
# root_directory = "./Results/" 
# data_directory = "/_Combined"
# prob_cal_from_all = False 
# data_subdir = ['STP_From_Same_Stages', 'STP_From_All_Stages'][int(prob_cal_from_all)] 
# wake_state_trimmed = False 
# data_subdirectory = f"Subject_One_Night{'_TrimW' if wake_state_trimmed else ''}"  ###"Subject_Combined_Record"  "Subject_Separate_Record"  "Subject_One_Night" "Subject_One_Night_TrimW"   ## Change for new type of result 
# # result_directory = f"./Results/_Classification/{result_subdir}" 
# result_directory = "./Results/_Classification" 
# metadata_subdirectory = ["CAP_Sleep", "Sleep_EDFX", "SDRC"] 
# dataset_list = ["CAP_Sleep", "Sleep_EDFX", "SDRC"] 
# tran_matrix_type = ["count", "dura", "proba"] 
# annotation_type = ['annot', 'tran']
# tran_step = 2
# exclude_contents_in_dataset_directory = ["SHA256SUMS", "RECORDS"]
# exclude_contents_in_result_directory = ["SHA256SUMS", "RECORDS", "all_annotations", "annot_sequence", "transition_sequence", "hypno", "DATASET_CHANGELOG"]
# sleep_stage_labels = ['W', 'S1', 'S2', 'S3', 'S4', 'REM']
# sleep_stage_labels_dict = {'W':0, 'S1':1, 'S2':2, 'S3':3, 'S4':4, 'REM':5}
# sleep_stage_names_dict = {'W':0, 'S1':1, 'S2':2, 'S3':3, 'S4':4, 'R':5}
# list_of_paths = None 


# info_type="sub" ##"sub"/"file"  ## Change for new type of result
# annot_type = annotation_type[0]   ## Change for different data preparation for 'annot' and 'tran' 


# directory = dataset_directory
# # list_of_paths = get_list_of_paths_from_a_directory(directory, path_type=None, containes=None, extension=None, exclude=None) 
# # list_of_paths = get_list_of_paths_from_a_directory(directory, path_type="p_file", containes=None, extension=None, exclude=None) 
# # list_of_paths = get_list_of_paths_from_a_directory(directory, path_type="p_dir", containes=None, extension=None, exclude=None) 
# # list_of_paths = get_list_of_paths_from_a_directory(directory, path_type=None, containes="nfle", extension=None, exclude=None) 
# # list_of_paths = get_list_of_paths_from_a_directory(directory, path_type=None, containes=None, extension="edf", exclude=None) 
# list_of_paths = get_list_of_paths_from_a_directory(directory, path_type=None, containes=None, extension=None, exclude=exclude_contents_in_dataset_directory) 
# # list_of_paths = get_list_of_paths_from_a_directory(directory, path_type=None, containes=None, extension=None, exclude=None) 
# # pprint(list_of_paths)
# list_of_paths

### Get basic information

In [6]:
all_demography_df

,#,Dataset,Category_Name,Category,Total_Count,Male_Count,Female_Count,Total_AgeRange,Male_AgeRange,Female_AgeRange
0,1,CAP_Sleep,Bruxism,brux,2,2,0,23 - 34,23 - 34,0 - 0
1,2,CAP_Sleep,Sleep-Disordered Breathing,sdb,4,4,0,65 - 78,65 - 78,0 - 0
2,3,CAP_Sleep,Insomnia,ins,9,4,5,47 - 82,54 - 82,47 - 59
3,4,CAP_Sleep,Narcolepsy,narco,5,2,3,18 - 44,24 - 43,18 - 44
4,5,CAP_Sleep,Nocturnal Frontal Lobe Epilepsy,nfle,40,21,19,14 - 67,14 - 44,16 - 67
5,6,CAP_Sleep,Periodic Leg Movements,plm,10,7,3,40 - 62,40 - 62,50 - 52
6,7,CAP_Sleep,REM Behavior Disorder,rbd,22,19,3,58 - 82,58 - 82,73 - 76
7,8,CAP_Sleep,No Pathology (Controls),n,16,7,9,23 - 42,23 - 34,24 - 42
8,10,CAP_Sleep,Total,NaN,108,66,42,14 - 82,14 - 82,16 - 76
9,11,CAP_Sleep,Sleep Disorders,dis,92,59,33,14 - 82,14 - 82,16 - 76


In [7]:
all_demography_detail_df

,Dataset,File_Name,Subject_Name,Category,Subject_ID,Gender,Age
0,CAP_Sleep,brux1,brux1,brux,1,M,34
1,CAP_Sleep,brux2,brux2,brux,2,M,23
2,CAP_Sleep,sdb1,sdb1,sdb,1,M,65
3,CAP_Sleep,sdb2,sdb2,sdb,2,M,77
4,CAP_Sleep,sdb3,sdb3,sdb,3,M,78
...,...,...,...,...,...,...,...
322,SDRC,ins1107,ins1107,ins,7,F,62
323,SDRC,ins1108,ins1108,ins,8,F,57
324,SDRC,ins1109,ins1109,ins,9,F,32
325,SDRC,ins1110,ins1110,ins,10,M,41


### Get transition matrix or features information from transition probabilities and P/AUC information

In [8]:
tran_mat_type = ['Proba', 'Dura', 'Count']
mat_info_type = 0 
print( f"{data_directory}/{result_subdirectory}/{annot_type.capitalize()}_{tran_mat_type[mat_info_type]}_Transition{tran_step}.csv" ) 
print( f"{data_directory}/{result_subdirectory}/{annot_type.capitalize()}_{tran_mat_type[mat_info_type]}_Transition{tran_step}_STAT_bin.csv" ) 
# print( f"{data_directory}/{result_subdirectory}/Annot_{tran_mat_type[mat_info_type]}_Transition{tran_step}.csv" ) Annot_Proba_Transition3_STAT_bin
# data_directory, result_subdirectory, tran_mat_type, mat_info_type, tran_step 

./Results//_Combined/STP_From_Same_Stages/Subject_One_Night_TrimW/Annot_Proba_Transition3.csv
./Results//_Combined/STP_From_Same_Stages/Subject_One_Night_TrimW/Annot_Proba_Transition3_STAT_bin.csv


In [9]:
def get_all_STP_matrix_files_for_PhDStudy01(mat_info_type, tran_step): 
    tran_mat_type = ['Proba', 'Dura', 'Count']
    ## ['EDFXSCRem', 'CAPOnly', 'CAPandSDRC']
    
    annot_proba_file_list = [] 
    annot_proba_file_list.append(f'./Results//_Combined/STP_From_Same_Stages/Subject_One_Night/Annot_{tran_mat_type[mat_info_type]}_Transition{tran_step}.csv') 
    annot_proba_file_list.append(f'./Results//_Combined/STP_From_Same_Stages/Subject_One_Night_TrimW/Annot_{tran_mat_type[mat_info_type]}_Transition{tran_step}.csv') 
    annot_proba_file_list.append(f'./Results//_Combined/STP_From_All_Stages/Subject_One_Night/Annot_{tran_mat_type[mat_info_type]}_Transition{tran_step}.csv') 
    annot_proba_file_list.append(f'./Results//_Combined/STP_From_All_Stages/Subject_One_Night_TrimW/Annot_{tran_mat_type[mat_info_type]}_Transition{tran_step}.csv') 
    tran_proba_file_list = [] 
    tran_proba_file_list.append(f'./Results//_Combined/STP_From_Same_Stages/Subject_One_Night/Tran_{tran_mat_type[mat_info_type]}_Transition{tran_step}.csv') 
    tran_proba_file_list.append(f'./Results//_Combined/STP_From_Same_Stages/Subject_One_Night_TrimW/Tran_{tran_mat_type[mat_info_type]}_Transition{tran_step}.csv') 
    tran_proba_file_list.append(f'./Results//_Combined/STP_From_All_Stages/Subject_One_Night/Tran_{tran_mat_type[mat_info_type]}_Transition{tran_step}.csv') 
    tran_proba_file_list.append(f'./Results//_Combined/STP_From_All_Stages/Subject_One_Night_TrimW/Tran_{tran_mat_type[mat_info_type]}_Transition{tran_step}.csv') 
    return annot_proba_file_list, tran_proba_file_list

In [10]:
def get_dataset_for_file_conditions2(mat_type, STP_From_All, TrimW, mat_info_type, tran_step):
    print(f'Reading criteria: {mat_type}, {STP_From_All}, {TrimW}, {mat_info_type}, {tran_step}')
    annot_proba_file_list, tran_proba_file_list = get_all_STP_matrix_files_for_PhDStudy01(mat_info_type, tran_step) 
    expected_list = annot_proba_file_list if mat_type==0 else tran_proba_file_list 
    ind = 0 
    if STP_From_All==1 and TrimW==1:
        ind = 3
    elif STP_From_All==1 and TrimW==0:
        ind = 2 
    elif STP_From_All==0 and TrimW==1:
        ind = 1
    else:
        ind = 0 
        
    filename = expected_list[ind] 
    print(f'Reading data from the file: {ind}, {filename}') 
    df = pd.read_csv(filename, index_col=False)
    return df

In [11]:
def get_dataset_for_file_conditions(mat_type, STP_From_All, TrimW, mat_info_type, tran_step):
    print(f'Reading criteria: {mat_type}, {STP_From_All}, {TrimW}, {mat_info_type}, {tran_step}')
    
    filename = f"{data_directory}/{result_subdirectory}/{annot_type.capitalize()}_{tran_mat_type[mat_info_type]}_Transition{tran_step}.csv"  
    stat_filename = f"{data_directory}/{result_subdirectory}/{annot_type.capitalize()}_{tran_mat_type[mat_info_type]}_Transition{tran_step}_STAT_bin.csv" 
    
    print(f'Reading data from the file: {filename}\nStatistics from the file: {stat_filename}') 
    df = pd.read_csv(filename, index_col=False)
    stat_df = pd.read_csv(stat_filename, index_col=False)
    return df, stat_df

In [15]:
# class_name='Class' 
# mat_type = 0 ## annot_type  # mat_type=annot/tran -- 0/1 
# STP_From_All = 0 ## int(tran_info_cal_from_all)  # STP_From_All=same/all -- 0/1 
# TrimW = 1 ## int(wake_state_trimmed)  # TrimW=normal/trimmed -- 0/1 
# mat_info_type = 0 ## int(tran_info_type) # Proba/Dura/Count -- 0/1/2
# tran_step = 3 ## int (tran step) # one level=2 step, two level=3 step -- 2/3

# dataset, stat_dataset = get_dataset_for_file_conditions(mat_type, STP_From_All, TrimW, mat_info_type, tran_step=tran_step) # mat_type=annot/tran, STP_From_All=same/all, TrimW=normal/trimmed, mat_info_type=proba/dura/count, tran_step=2/3 
# dataset 

In [17]:
# dataset.isna().sum().sum()

#### Prepare final dataset

In [ ]:
# def map_age_category_to_class(dat_set, demo_det, source_cols=['Subject_Name', 'File_Name'], age_col = 'Age', age_ranges = [[19, 30], [31, 40]], class_name='Class', multi_class=True, healthy_only=True):
#     tmp_df = dataset.merge(all_demography_detail_df, how='inner', left_on=[source_cols[0]], right_on=[source_cols[1]])
#     tmp_df

#     dat_set.insert(3, age_col, tmp_df[age_col].values) 
#     dat_set

#     dat_set = dat_set[ ((dat_set[age_col]>=age_ranges[0][0]) & (dat_set[age_col]<=age_ranges[0][1])) | ((dat_set[age_col]>=age_ranges[1][0]) & (dat_set[age_col]<=age_ranges[1][1])) ]  
#     dat_set.reset_index(drop=True, inplace=True)

#     tmp_age_range = [list(range(age_ranges[0][0], age_ranges[0][1]+1)), list(range(age_ranges[1][0], age_ranges[1][1]+1))] 
#     tmp_age_range 

#     cls_map = {} 
#     for i, lst in enumerate(tmp_age_range): 
#         for l in lst:
#             cls_map[l]=i
#     cls_map
    
#     all_cols = dat_set.columns.values.tolist() 
#     if (age_col in all_cols):
#         if (class_name in all_cols): 
#             dat_set = dat_set.drop([class_name], axis=1)
#         dat_set = dat_set.rename(columns={age_col: class_name})
    
#     all_cols2 = dat_set.columns.values.tolist() 
#     if (class_name not in all_cols) and (class_name in all_cols2):
#         dat_set.replace({class_name: cls_map}, inplace=True) 
        
#     if healthy_only==True: 
#         pass
#     return cls_map, dat_set 

In [ ]:
# all_demography_detail_df
# dataset

In [12]:
def map_age_category_to_class(dat_set, demo_det, source_cols=['Subject_Name', 'File_Name'], age_col = 'Age', age_ranges = [[19, 30], [31, 40]], class_name='Class', multi_class=True, healthy_only=True):
    if healthy_only==True: 
        dat_set = dat_set[(dat_set['Category']=='n')] 
    
    tmp_df = dat_set.merge(demo_det, how='inner', left_on=[source_cols[0]], right_on=[source_cols[1]])
    tmp_df
#     print(tmp_df.shape[0], dat_set.shape[0])
    assert tmp_df.shape[0]==dat_set.shape[0], "Something hapend. No shape after dataframe joining is same..."

    dat_set.insert(3, age_col, tmp_df[age_col].values) 
    dat_set
#     print(age_ranges, dat_set)

#     dat_set = dat_set[ ((dat_set[age_col]>=age_ranges[0][0]) & (dat_set[age_col]<=age_ranges[0][1])) | ((dat_set[age_col]>=age_ranges[1][0]) & (dat_set[age_col]<=age_ranges[1][1])) ]  
#     dat_set.reset_index(drop=True, inplace=True)
    
    tmp_age_range = []
    tmp_dat_set = pd.DataFrame() 
    for i, ar in enumerate(age_ranges):
        sel_df = dat_set[((dat_set[age_col]>=ar[0]) & (dat_set[age_col]<=ar[1]))] 
        tmp_dat_set = pd.concat([tmp_dat_set, sel_df ] )
        tmp_age_range.append( sorted(sel_df[age_col].values.tolist()) ) 
        tmp_age_range 
#         print(tmp_age_range, dat_set)
    dat_set = tmp_dat_set.copy()
    dat_set.reset_index(drop=True, inplace=True)
#     print(dat_set)

    cls_map = {} 
    for i, lst in enumerate(tmp_age_range): 
        for l in lst:
            cls_map[l]=i
    cls_map
    
    all_cols = dat_set.columns.values.tolist() 
    if (age_col in all_cols):
        if (class_name in all_cols): 
            dat_set = dat_set.drop([class_name], axis=1)
        dat_set = dat_set.rename(columns={age_col: class_name})
    
    all_cols2 = dat_set.columns.values.tolist() 
    if (class_name not in all_cols) and (class_name in all_cols2):
        dat_set.replace({class_name: cls_map}, inplace=True) 
    return cls_map, dat_set 

In [ ]:
# # label_map, dataset2 = map_age_category_to_class(dataset.copy(), all_demography_detail_df.copy(), source_cols=['Subject_Name', 'File_Name'], age_col = 'Age', age_ranges = [[19, 30], [31, 200]], class_name='Class', multi_class=True, healthy_only=True) ##19-101 
# label_map, dataset2 = map_age_category_to_class(dataset.copy(), all_demography_detail_df.copy(), source_cols=['Subject_Name', 'File_Name'], age_col = 'Age', age_ranges = [[1, 30], [40, 60], [70, 200]], class_name='Class', multi_class=True, healthy_only=True) ##19-101 
# print(label_map)
# dataset2

In [13]:
def map_category_to_class(dat_set, source_col='Category', class_name='Class', removable_cats=None, multi_class=True, age_data=None, age_col = 'Age', source_cols=['Subject_Name', 'File_Name']): 
    if class_name in dat_set.columns.tolist():
        dat_set = dat_set.drop(columns=[class_name])
    dat_set.insert(3, class_name, dat_set[source_col].values) 
    dat_set

    cat_val = dat_set[source_col].unique().tolist() 
    cat_val.remove('n')
    cat_val.insert(0, 'n')
    print(cat_val) 
    
    if removable_cats:
        cat_val = [c for c in cat_val if c not in removable_cats]
        dat_set = dat_set[dat_set[source_col].isin(cat_val)]
        dat_set.reset_index(drop=True, inplace=True)
    print(cat_val) 
        
    cls_map = dict(zip(cat_val, list(range(len(cat_val))))) 
    cls_map
    
    if not multi_class:
        for k in cls_map.keys():
            if cls_map[k]>1:
                cls_map[k]=1

    dat_set.replace({class_name: cls_map}, inplace=True) 
    
    if age_data is not None: 
        tmp_df = dat_set.merge(age_data, how='inner', left_on=[source_cols[0]], right_on=[source_cols[1]])
        tmp_df
        assert tmp_df.shape[0]==dat_set.shape[0], "Something hapend. No shape after dataframe joining is same..." 
        sel_ind = dat_set.columns.values.tolist().index(class_name)+1 
        dat_set.insert(sel_ind, age_col, tmp_df[age_col].values)
    
    return cls_map, dat_set 
    
    

In [ ]:
# label_map, dataset2 = map_category_to_class(dataset.copy(), source_col='Category', class_name='Class', removable_cats=None, multi_class=False, age_data=all_demography_detail_df.copy(), age_col = 'Age', source_cols=['Subject_Name', 'File_Name'])
# print(label_map)
# dataset2

#### Get features with zeo transition

In [20]:
def get_features_with_zero_values(tmp_df):
    feat_names = tmp_df.columns.values[4:] 
    feat_names
    zero_feats = [] 
    for f in feat_names:
        if tmp_df[f].values.sum() == 0:
            zero_feats.append(f)
    return zero_feats

# Classification 

In [14]:
def create_experiment_directory(path, exp_name): 
    exp_directory = f"{path}/{exp_name}/"

    if (not os.path.exists(exp_directory)):
        try:
            os.makedirs(exp_directory, exist_ok = True)
            print(f"Directory successfully created at path: {exp_directory}") 
        except OSError as error:
            print(f"Directory cannot be created at path: {exp_directory}") 
    else:
        print(f"Directory already exists at path: {exp_directory}") 

    return exp_directory

In [15]:
ML_Classifiers.SVC, ML_Classifiers.XGBoost

(<ML_Classifiers.SVC: 'support_vector_classifier'>,
 <ML_Classifiers.XGBoost: 'xtreme_gradient_boosting'>)

In [16]:
__name__, os.getcwd()

('__main__',
 'C:\\Users\\aliem\\OneDrive - Deakin University\\_MyResearch\\PhD_Research\\HML_IHC_Sleep_Data_Analysis')

In [17]:
def start_logger(result_save_path, exp_name): 
    util = Humachlab_Utility() 
    all_log_file_name = f'{result_save_path}/all_logs_{exp_name}.txt'
    # logger = util.get_logger(logger_name=__name__, log_file_name=all_log_file_name)
    logger = util.get_logger(logger_name="Sleep ML Model Analysis", log_file_name=all_log_file_name)
    util, all_log_file_name, logger 
    return util, logger

In [18]:
def stop_logger(logger): 
    handlers = logger.handlers[:]
    for handler in handlers:
        logger.removeHandler(handler)
        handler.close()

In [19]:
def modify_experiment_information_summarry(exp_dir, dict_dat=None):
    exp_sum_dir = f'{exp_dir}/Experiment_Information.csv'
    df = pd.DataFrame(columns=['exp_name', 'stp_from', 'exp_description', 'datasets', 'feature_selection', 'special_consideration', 'classification_type'])
    if (os.path.exists(exp_sum_dir)):
        df = pd.read_csv(exp_sum_dir, index_col=False) 
    if dict_dat:
        if df.shape[0]>0:
            print(dict_dat['exp_name'], (df['exp_name'].values.tolist()))
            if dict_dat['exp_name'] in (df['exp_name'].values.tolist()):
                # nn = 1 
                # print(f'Cannot add this data. this experiment is already exited. Please recheck and do a new experiment')
                # assert nn < 0, f'Cannot add this data. this experiment is already exited. Please recheck and do a new experiment' 
                print(f'This experiment is already exited. So this data is removed...')
                df = df.loc[df["exp_name"] != dict_dat['exp_name']] 
                df.reset_index(drop=True, inplace=True) 
                
        df.loc[len(df)] = dict_dat
        df.sort_values(by=['exp_name'], ascending=[True], inplace=True)
        df.reset_index(drop=True, inplace=True) 
        df.to_csv(exp_sum_dir, index=False) 
        print(f'Data is successfully inserted...')
    return df

In [20]:
class_name='Class' 
mat_type = 0 ## annot_type  # mat_type=annot/tran -- 0/1 
STP_From_All = 0 ## int(tran_info_cal_from_all)  # STP_From_All=same/all -- 0/1 
TrimW = 1 ## int(wake_state_trimmed)  # TrimW=normal/trimmed -- 0/1 
mat_info_type = 0 ## int(tran_info_type) # Proba/Dura/Count -- 0/1/2
tran_step = 2 ## int (tran step) # one level=2 step, two level=3 step -- 2/3

# ##dataset, stat_dataset = get_dataset_for_file_conditions(mat_type, STP_From_All, TrimW, mat_info_type, tran_step=tran_step) # mat_type=annot/tran, STP_From_All=same/all, TrimW=normal/trimmed, mat_info_type=proba/dura/count, tran_step=2/3 
# ##dataset 

# dataset1, stat_dataset1 = get_dataset_for_file_conditions(mat_type, STP_From_All, TrimW, mat_info_type, tran_step=tran_step) # mat_type=annot/tran, STP_From_All=same/all, TrimW=normal/trimmed, mat_info_type=proba/dura/count 
# dataset1 

dataset2, stat_dataset2 = get_dataset_for_file_conditions(mat_type, STP_From_All, TrimW, mat_info_type, tran_step=tran_step) # mat_type=annot/tran, STP_From_All=same/all, TrimW=normal/trimmed, mat_info_type=proba/dura/count 
dataset2 

Reading criteria: 0, 0, 1, 0, 2
Reading data from the file: ./Results//_Combined/STP_From_Same_Stages/Subject_One_Night_TrimW/Annot_Proba_Transition2.csv
Statistics from the file: ./Results//_Combined/STP_From_Same_Stages/Subject_One_Night_TrimW/Annot_Proba_Transition2_STAT_bin.csv


,Dataset,Category,Subject_Name,W->W,W->S1,W->S2,W->S3,W->S4,W->REM,S1->W,...,S4->S2,S4->S3,S4->S4,S4->REM,REM->W,REM->S1,REM->S2,REM->S3,REM->S4,REM->REM
0,CAP_Sleep,brux,brux1,0.798077,0.192308,0.009615,0.0,0.0,0.000000,0.030769,...,0.004926,0.024631,0.970443,0.0,0.033520,0.000000,0.000000,0.0,0.0,0.966480
1,CAP_Sleep,brux,brux2,0.824000,0.176000,0.000000,0.0,0.0,0.000000,0.061728,...,0.000000,0.012048,0.984940,0.0,0.019324,0.004831,0.000000,0.0,0.0,0.975845
2,CAP_Sleep,sdb,sdb1,0.876147,0.123853,0.000000,0.0,0.0,0.000000,0.099237,...,0.005076,0.030457,0.949239,0.0,0.009346,0.000000,0.028037,0.0,0.0,0.962617
3,CAP_Sleep,sdb,sdb2,0.948387,0.045161,0.006452,0.0,0.0,0.000000,0.011364,...,0.008475,0.016949,0.966102,0.0,0.000000,0.000000,0.025641,0.0,0.0,0.974359
4,CAP_Sleep,sdb,sdb3,0.934272,0.046948,0.018779,0.0,0.0,0.000000,0.108696,...,0.025974,0.038961,0.935065,0.0,0.062500,0.000000,0.000000,0.0,0.0,0.937500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,Sleep_EDFX,n,ST7191,0.739130,0.246377,0.014493,0.0,0.0,0.000000,0.044118,...,0.013889,0.166667,0.805556,0.0,0.000000,0.020000,0.015000,0.0,0.0,0.965000
226,Sleep_EDFX,n,ST7201,0.294118,0.588235,0.058824,0.0,0.0,0.058824,0.014493,...,0.038462,0.615385,0.346154,0.0,0.044776,0.007463,0.014925,0.0,0.0,0.932836
227,Sleep_EDFX,n,ST7211,0.894118,0.100000,0.000000,0.0,0.0,0.005882,0.116667,...,0.016949,0.254237,0.728814,0.0,0.025641,0.000000,0.019231,0.0,0.0,0.955128
228,Sleep_EDFX,n,ST7221,0.818792,0.181208,0.000000,0.0,0.0,0.000000,0.052133,...,0.000000,0.000000,0.000000,0.0,0.021645,0.043290,0.000000,0.0,0.0,0.935065


In [90]:
# dataset1.shape, dataset2.shape

((230, 39), (230, 219))

In [29]:
# [i for i in dataset2.Subject_Name.values.tolist() if i not in dataset1.Subject_Name.values.tolist()]

In [98]:
# inner_merged_total = pd.merge(
#     dataset1, dataset2, on=['Dataset', 'Category', 'Subject_Name'], suffixes=['_proba', '_dura']
# )
# inner_merged_total

# inner_merged_total = pd.merge(
#     dataset1, dataset2, on=['Dataset', 'Category', 'Subject_Name'], suffixes=['_proba', '_dura']
# )
# print(inner_merged_total.columns.values) 

In [30]:
# pd.concat([stat_dataset1, stat_dataset2], axis=0) 

In [21]:
# dataset = pd.concat([dataset1.add_prefix('proba_'), dataset2.add_prefix('dura_')], axis=1)
# dataset = pd.merge(dataset1, dataset2, on=['Dataset', 'Category', 'Subject_Name'], suffixes=['_proba', '_dura'])
# dataset = pd.merge(dataset1, dataset2, on=['Dataset', 'Category', 'Subject_Name'], suffixes=['_annot', '_tran'])

# dataset = dataset1.copy()
# stat_dataset = stat_dataset1.copy() 
dataset = dataset2.copy()
stat_dataset = stat_dataset2.copy() 

# dataset = pd.merge(
#     dataset1, dataset2, on=['Dataset', 'Category', 'Subject_Name'], suffixes=['_proba', '_dura']
# )
# stat_dataset = pd.concat([stat_dataset1, stat_dataset2], axis=0) 

print(stat_dataset.columns.values) 
dataset

['Features' 'Sum' 'Mean' 'STD' 'Healthy_Sum' 'Healthy_Mean' 'Healthy_STD'
 'Disorder_Sum' 'Disorder_Mean' 'Disorder_STD' 'P_Value' 'AUC'
 'Wilcoxon_zscore' 'Wilcoxon_pvalue' 'MannWhitney_statistic'
 'MannWhitney_pvalue']


,Dataset,Category,Subject_Name,W->W,W->S1,W->S2,W->S3,W->S4,W->REM,S1->W,...,S4->S2,S4->S3,S4->S4,S4->REM,REM->W,REM->S1,REM->S2,REM->S3,REM->S4,REM->REM
0,CAP_Sleep,brux,brux1,0.798077,0.192308,0.009615,0.0,0.0,0.000000,0.030769,...,0.004926,0.024631,0.970443,0.0,0.033520,0.000000,0.000000,0.0,0.0,0.966480
1,CAP_Sleep,brux,brux2,0.824000,0.176000,0.000000,0.0,0.0,0.000000,0.061728,...,0.000000,0.012048,0.984940,0.0,0.019324,0.004831,0.000000,0.0,0.0,0.975845
2,CAP_Sleep,sdb,sdb1,0.876147,0.123853,0.000000,0.0,0.0,0.000000,0.099237,...,0.005076,0.030457,0.949239,0.0,0.009346,0.000000,0.028037,0.0,0.0,0.962617
3,CAP_Sleep,sdb,sdb2,0.948387,0.045161,0.006452,0.0,0.0,0.000000,0.011364,...,0.008475,0.016949,0.966102,0.0,0.000000,0.000000,0.025641,0.0,0.0,0.974359
4,CAP_Sleep,sdb,sdb3,0.934272,0.046948,0.018779,0.0,0.0,0.000000,0.108696,...,0.025974,0.038961,0.935065,0.0,0.062500,0.000000,0.000000,0.0,0.0,0.937500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,Sleep_EDFX,n,ST7191,0.739130,0.246377,0.014493,0.0,0.0,0.000000,0.044118,...,0.013889,0.166667,0.805556,0.0,0.000000,0.020000,0.015000,0.0,0.0,0.965000
226,Sleep_EDFX,n,ST7201,0.294118,0.588235,0.058824,0.0,0.0,0.058824,0.014493,...,0.038462,0.615385,0.346154,0.0,0.044776,0.007463,0.014925,0.0,0.0,0.932836
227,Sleep_EDFX,n,ST7211,0.894118,0.100000,0.000000,0.0,0.0,0.005882,0.116667,...,0.016949,0.254237,0.728814,0.0,0.025641,0.000000,0.019231,0.0,0.0,0.955128
228,Sleep_EDFX,n,ST7221,0.818792,0.181208,0.000000,0.0,0.0,0.000000,0.052133,...,0.000000,0.000000,0.000000,0.0,0.021645,0.043290,0.000000,0.0,0.0,0.935065


In [32]:
# pd.concat?

In [33]:
dataset.isna().sum().sum() 

0

In [68]:
dataset['Dataset'].unique() 

array(['CAP_Sleep', 'SDRC', 'Sleep_EDFX'], dtype=object)

In [35]:
dataset['Subject_Name'].unique().shape

(230,)

In [22]:
zero_feats = get_features_with_zero_values(dataset)
print(len(zero_feats), zero_feats) 

NameError: name 'get_features_with_zero_values' is not defined

In [37]:
stat_dataset

,Features,Sum,Mean,STD,Healthy_Sum,Healthy_Mean,Healthy_STD,Disorder_Sum,Disorder_Mean,Disorder_STD,P_Value,AUC,Wilcoxon_zscore,Wilcoxon_pvalue,MannWhitney_statistic,MannWhitney_pvalue
0,W->W,200.023853,0.869669,0.111130,108.678140,0.855733,0.120193,91.345712,0.886852,0.096074,3.481301e-02,0.403218,-2.522887,1.163957e-02,5274.5,5.836077e-03
1,W->S1,24.969913,0.108565,0.094053,15.941897,0.125527,0.099497,9.028017,0.087651,0.082186,2.277298e-03,0.347374,3.978629,6.931372e-05,4544.0,3.479935e-05
2,W->S2,4.074743,0.017716,0.030857,1.629452,0.012830,0.027523,2.445291,0.023741,0.033563,7.522503e-03,0.653811,-4.009518,6.084289e-05,4528.5,1.431124e-05
3,W->S3,0.219641,0.000955,0.004911,0.108293,0.000853,0.004927,0.111348,0.001081,0.004890,7.272645e-01,0.511199,-0.291945,7.703283e-01,6394.0,2.483123e-01
4,W->S4,0.125777,0.000547,0.007757,0.117647,0.000926,0.010398,0.008130,0.000079,0.000797,4.121993e-01,0.500879,-0.022917,9.817163e-01,6529.0,4.457874e-01
5,W->REM,0.586073,0.002548,0.009081,0.524571,0.004130,0.011837,0.061502,0.000597,0.002123,3.217895e-03,0.422139,2.029669,4.239021e-02,5522.0,1.180169e-03
6,S1->W,15.246232,0.066288,0.067213,9.969784,0.078502,0.053088,5.276448,0.051228,0.078752,2.102759e-03,0.700673,5.231105,1.684994e-07,3915.5,8.178827e-08
7,S1->S1,164.464191,0.715062,0.143797,85.942910,0.676716,0.142570,78.521282,0.762343,0.130624,4.897664e-06,0.686301,-4.854466,1.207117e-06,4104.5,6.063551e-07
8,S1->S2,44.629370,0.194041,0.112006,27.225130,0.214371,0.116463,17.404240,0.168973,0.100755,2.128943e-03,0.381011,3.101796,1.923502e-03,4984.0,9.646592e-04
9,S1->S3,1.493990,0.006496,0.026515,0.857565,0.006752,0.024687,0.636425,0.006179,0.028607,8.711021e-01,0.505695,0.148464,8.819768e-01,6466.0,3.975669e-01


In [23]:
def get_top_features_baseOn_pValue(tstat_dataset, pval=0.05): 
    tsdf = tstat_dataset[(tstat_dataset['P_Value']<=pval)] 
    return tsdf 

def get_top_features_baseOn_AUC(tstat_dataset, auc=0.5): 
    tsdf = tstat_dataset[(tstat_dataset['AUC']>auc)] 
    return tsdf 

In [24]:
def get_top_featute_names_baseOn_pValue(tstat_dataset, pval=0.05): 
    top_feat_df = get_top_features_baseOn_pValue(tstat_dataset, pval=pval)
    top_feat_df
    return top_feat_df['Features'].values.tolist() 

def get_top_featute_names_baseOn_AUC(tstat_dataset, auc=0.5): 
    top_feat_df = get_top_features_baseOn_AUC(tstat_dataset, auc=auc)
    top_feat_df
    return top_feat_df['Features'].values.tolist() 

In [25]:
top_feats_p = get_top_featute_names_baseOn_pValue(stat_dataset, pval=0.05)
len(top_feats_p), top_feats_p

(15,
 ['W->W',
  'W->S1',
  'W->S2',
  'W->REM',
  'S1->W',
  'S1->S1',
  'S1->S2',
  'S1->REM',
  'S2->S3',
  'S3->S2',
  'S3->S3',
  'S3->S4',
  'S4->S3',
  'S4->S4',
  'REM->S1'])

In [26]:
top_feats_a = get_top_featute_names_baseOn_AUC(stat_dataset, auc=0.6)
len(top_feats_a), top_feats_a

(11,
 ['W->S2',
  'S1->W',
  'S1->S1',
  'S2->W',
  'S2->S1',
  'S2->S3',
  'S2->REM',
  'S3->S2',
  'S3->S3',
  'S4->S2',
  'S4->S4'])

In [39]:
# dataset['Dataset'].unique()

In [192]:
# dataset[~(dataset['Subject_Name'].str.startswith('SC'))]
# dataset[~(dataset['Dataset']=='Sleep_EDFX')]
dataset

,Dataset,Category,Subject_Name,W->W,W->S1,W->S2,W->S3,W->S4,W->REM,S1->W,...,S4->S2,S4->S3,S4->S4,S4->REM,REM->W,REM->S1,REM->S2,REM->S3,REM->S4,REM->REM
0,CAP_Sleep,brux,brux1,0.798077,0.192308,0.009615,0.0,0.0,0.000000,0.030769,...,0.004926,0.024631,0.970443,0.0,0.033520,0.000000,0.000000,0.0,0.0,0.966480
1,CAP_Sleep,brux,brux2,0.824000,0.176000,0.000000,0.0,0.0,0.000000,0.061728,...,0.000000,0.012048,0.984940,0.0,0.019324,0.004831,0.000000,0.0,0.0,0.975845
2,CAP_Sleep,sdb,sdb1,0.876147,0.123853,0.000000,0.0,0.0,0.000000,0.099237,...,0.005076,0.030457,0.949239,0.0,0.009346,0.000000,0.028037,0.0,0.0,0.962617
3,CAP_Sleep,sdb,sdb2,0.948387,0.045161,0.006452,0.0,0.0,0.000000,0.011364,...,0.008475,0.016949,0.966102,0.0,0.000000,0.000000,0.025641,0.0,0.0,0.974359
4,CAP_Sleep,sdb,sdb3,0.934272,0.046948,0.018779,0.0,0.0,0.000000,0.108696,...,0.025974,0.038961,0.935065,0.0,0.062500,0.000000,0.000000,0.0,0.0,0.937500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,Sleep_EDFX,n,ST7191,0.739130,0.246377,0.014493,0.0,0.0,0.000000,0.044118,...,0.013889,0.166667,0.805556,0.0,0.000000,0.020000,0.015000,0.0,0.0,0.965000
226,Sleep_EDFX,n,ST7201,0.294118,0.588235,0.058824,0.0,0.0,0.058824,0.014493,...,0.038462,0.615385,0.346154,0.0,0.044776,0.007463,0.014925,0.0,0.0,0.932836
227,Sleep_EDFX,n,ST7211,0.894118,0.100000,0.000000,0.0,0.0,0.005882,0.116667,...,0.016949,0.254237,0.728814,0.0,0.025641,0.000000,0.019231,0.0,0.0,0.955128
228,Sleep_EDFX,n,ST7221,0.818792,0.181208,0.000000,0.0,0.0,0.000000,0.052133,...,0.000000,0.000000,0.000000,0.0,0.021645,0.043290,0.000000,0.0,0.0,0.935065


In [196]:
dataset['Dataset'].unique()

array(['CAP_Sleep', 'SDRC', 'Sleep_EDFX'], dtype=object)

In [28]:
print( f"{data_directory}/{result_subdirectory}/{annot_type.capitalize()}_{tran_mat_type[mat_info_type]}_Transition{tran_step}.csv" ) 
print( f"{data_directory}/{result_subdirectory}/{annot_type.capitalize()}_{tran_mat_type[mat_info_type]}_Transition{tran_step}_STAT_bin.csv" ) 

./Results//_Combined/STP_From_Same_Stages/Subject_One_Night_TrimW/Annot_Proba_Transition2.csv
./Results//_Combined/STP_From_Same_Stages/Subject_One_Night_TrimW/Annot_Proba_Transition2_STAT_bin.csv


In [29]:
exp_sum_df = modify_experiment_information_summarry(result_directory) 
exp_sum_df 

# exp_sum_df = modify_experiment_information_summarry(result_directory, dict_dat=exp_detail) 
# exp_sum_df 

# exp_sum_df = modify_experiment_information_summarry(result_directory) 
# exp_sum_df 

,exp_name,stp_from,exp_description,datasets,feature_selection,special_consideration,classification_type
0,ML0011,Same,Data classification using sleep transition mat...,"2 datasets- CAP_Sleep, Sleep_EDFX",No feature selection,"Remove all zero, subject balanced over the fol...",Binary classification- Healthy vs disordered
1,ML0012,Same,Data classification using sleep transition mat...,"2 datasets- CAP_Sleep, Sleep_EDFX",No feature selection,"Remove all zero, subject balanced over the fol...",Binary classification- Healthy vs disordered
2,ML0013,Same,Data classification using sleep transition mat...,"2 datasets- CAP_Sleep, Sleep_EDFX",No feature selection,"Remove all zero, subject balanced over the fol...",Binary classification- Healthy vs disordered
3,ML10001,Same,Data classification using sleep transition mat...,2 datasets- All,AUC>0.5,"Remove all zero transition and W->W stage, sub...",Binary classification- Healthy vs disordered
4,ML10002,Same,Data classification using sleep transition mat...,2 datasets- All,AUC>0.5,"Remove all zero, subject balanced over the fol...",Binary classification- Healthy vs disordered
...,...,...,...,...,...,...,...
103,ML6005,Same,Data classification using sleep transition mat...,3 datasets- All,No feature selection,"Remove all zero, AUC>0.5, subject balanced ove...",Binary classification- Healthy vs disordered
104,ML6006,Same,Data classification using sleep transition mat...,3 datasets- All,No feature selection,"Remove all zero, AUC>0.6, subject balanced ove...",Binary classification- Healthy vs disordered
105,ML6007,Same,Data classification using sleep transition mat...,3 datasets- All,No feature selection,"Remove all zero, pval<=0.05, subject balanced ...",Binary classification- Healthy vs disordered
106,ML6008,Same,Data classification using sleep transition mat...,3 datasets- All,No feature selection,"Remove all zero, AUC>0.5, subject balanced ove...",Binary classification- Healthy vs disordered


In [30]:
# pd.read_csv?
# exp_detail
# exp_name
try:
    stop_logger(logger)
except Exception as e:
    print(e)
    logger = None
    
# logger = None

# del logger

name 'logger' is not defined


In [31]:
# tran_type
tran_step

2

In [182]:
exp_name = 'ML10001'
exp_name = 'ML4001'
exp_name = 'ML10008'
result_save_path = create_experiment_directory(result_directory, exp_name) 
print(result_save_path) 

exp_info_path = f"{result_save_path}all_exp_info.csv" 
print(exp_info_path)

exp_info_df = pd.read_csv(exp_info_path) 
exp_info_df 

top_feats = exp_info_df[(exp_info_df['Information']=='dataset_columns')]['Description'].values[0] 
top_feats = top_feats.replace('[', '').replace(']', '').split(', ') 
top_feats = [c.replace('\'', '') for c in top_feats] 
# print(top_feats) 
sel_cols = top_feats[top_feats.index(class_name)+1:] 
print(sel_cols) 
exp_info_df 

Directory already exists at path: ./Results//_Classification/ML10008/
./Results//_Classification/ML10008/
./Results//_Classification/ML10008/all_exp_info.csv
['W->S1', 'W->S2', 'W->S3', 'W->S4', 'W->REM', 'S1->W', 'S1->S1', 'S1->S2', 'S1->S3', 'S1->S4', 'S1->REM', 'S2->W', 'S2->S1', 'S2->S2', 'S2->S3', 'S2->S4', 'S2->REM', 'S3->W', 'S3->S1', 'S3->S2', 'S3->S3', 'S3->S4', 'S3->REM', 'S4->W', 'S4->S1', 'S4->S2', 'S4->S3', 'S4->S4', 'S4->REM', 'REM->W', 'REM->S1', 'REM->S2', 'REM->S3', 'REM->S4', 'REM->REM']


,Information,Description
0,logger,<Logger Sleep ML Model Analysis (DEBUG)>
1,directory,./Results//_Classification/ML10008/
2,dataset_size,"(47, 39)"
3,dataset_columns,"['Dataset', 'Category', 'Subject_Name', 'Class..."
4,metadata_column,"['Dataset', 'Category', 'Subject_Name']"
5,class_name,Class
6,label_map,"{'n': 0, 'ins': 1}"
7,split_column,Subject_Name
8,total_unique_classes,"[27, 20]"
9,exp_name,ML10008


In [183]:
# logger.info("Hello")
ML_Classifiers.SVC, ML_Classifiers.RF, ML_Classifiers.XGBoost

(<ML_Classifiers.SVC: 'support_vector_classifier'>,
 <ML_Classifiers.RF: 'random_forest'>,
 <ML_Classifiers.XGBoost: 'xtreme_gradient_boosting'>)

In [184]:
random_state_value = 312
class_name = "Class" 
metadata_column = ["Dataset", "Category", "Subject_Name"] 
all_metadata_columns = metadata_column+[class_name]
# # ### #Binary/Multi-class healthy vs disorders 
# split_column = "Subject_Name"  #"Subject_Name" for binary or multi-class 
# split_balance_pattern = [['n'], ['SC', 'ST'], ['brux'], ['sdb'], ['ins'], ['narco'], ['nfle'], ['plm'], ['rbd']] #[['n'], ['SC', 'ST']]  for binary || [['n'], ['SC', 'ST'], ['brux'], ['sdb'], ['ins'], ['narco'], ['nfle'], ['plm'], ['rbd']] for multi-class 
# # split_balance_pattern = [['n'], ['ST'], ['brux'], ['sdb'], ['ins'], ['narco'], ['nfle'], ['plm'], ['rbd']] #[['n'], ['SC', 'ST']]  for binary || [['n'], ['SC', 'ST'], ['brux'], ['sdb'], ['ins'], ['narco'], ['nfle'], ['plm'], ['rbd']] for multi-class 
# # split_balance_pattern = [['n'], ['brux'], ['sdb'], ['ins'], ['narco'], ['nfle'], ['plm'], ['rbd']] #[['n'], ['SC', 'ST']]  for binary || [['n'], ['SC', 'ST'], ['brux'], ['sdb'], ['ins'], ['narco'], ['nfle'], ['plm'], ['rbd']] for multi-class 
# ### #Binary/Multi-class age-group detection  
# # split_column = class_name
# # split_balance_pattern = [[1]]
class_name, metadata_column, all_metadata_columns, result_save_path


('Class',
 ['Dataset', 'Category', 'Subject_Name'],
 ['Dataset', 'Category', 'Subject_Name', 'Class'],
 './Results//_Classification/ML10008/')

In [185]:
#### 2 step transition feature selection 
# dataset
# OLD ['S3->S3','S4->S4','S3->S2','W->S1','W->S2','S2->S2','REM->REM','S4->S3','S3->S4','S2->S1','S2->S3','S2->W']

### From 1031 
# AUC_0_7 = ['S3->S3','W->S1','S3->S4','S3->S2','S4->S4','S2->S2','S4->S3','W->S2','S2->S3','S1->W','S2->S1'] 
# AUC_0_5 = ['S3->S3','W->S1','S3->S4','S3->S2','S4->S4','S2->S2','S4->S3','W->S2','S2->S3','S1->W','S2->S1','S3->W','S1->REM','S1->S1','REM->REM','S2->REM','S4->S2','REM->W',
#            'REM->S1','S2->W','S1->S2','S4->W','S3->S1','W->S3','W->REM','S2->S4','S3->REM']
# RF_FI_0_2 = ['S3->S3','S4->S4','S3->S2','W->S1','S2->S2','W->S2','S4->S3','S2->S1','S3->S4','S1->S1','S2->S3','REM->REM','S2->W','S2->REM','REM->W'] 

# ### Overall AUC
# AUC_0_7 = ['W->W', 'S1->W', 'S1->REM', 'S2->S1', 'S2->S2', 'S3->S3', 'S4->S4']
# AUC_0_5 = ['W->W', 'W->S2', 'S1->W', 'S1->REM', 'S2->S1', 'S2->S2', 'S2->S4', 'S2->REM', 'S3->S2', 'S3->S3', 'S4->W', 'S4->S2', 'S4->S3', 'S4->S4', 'REM->S2']

### From 1135  
AUC_0_7 = ['S3->S3', 'S4->S4', 'S2->S2', 'S4->S3', 'REM->REM', 'S3->S4', 'S2->REM', 'S3->S2', 'S2->S1', 'W->S2', 'S1->REM'] 
AUC_0_5 = ['S3->S3', 'S4->S4', 'S2->S2', 'S4->S3', 'REM->REM', 'S3->S4', 'S2->REM', 'S3->S2', 'S2->S1', 'W->S2', 'S1->REM', 'S4->S2', 'S1->W', 'S4->W', 'REM->S2', 
           'S2->W', 'S2->S3', 'REM->S1', 'S3->W', 'S2->S4', 'S1->S1', 'W->S3'] 
RF_FI_0_2 = ['S3->S3','S4->S4','S2->S2','REM->REM','S3->S2','S3->S4','S2->S1','S1->REM','S4->S3','S1->S2','S2->S3','W->S2','W->S1','S4->S2','S2->REM','REM->S2','S1->W','S1->S1','S4->W'] 

theoritically_feasible_transitions = ['W->S1', 'W->S2', 'S1->W', 'S1->S1', 'S1->S2', 'S2->W', 'S2->S1', 'S2->S2', 'S2->S3', 'S2->S4', 'S2->REM', 'S3->W', 'S3->S2', 'S3->S3', 'S3->S4',
                                     'S4->W', 'S4->S2', 'S4->S3', 'S4->S4', 'REM->W', 'REM->S2', 'REM->REM'] 
feats_cant_seperate_both_healthy = ['W->W', 'W->S1', 'W->REM', 'S1->W', 'S1->S1', 'S1->REM', 'S2->W', 'S2->S1', 'S2->S2', 'S2->S3', 'S2->REM', 'S3->S2', 'S3->S3', 'S4->S2', 'S4->S4', 
                                    'REM->W', 'REM->S1', 'REM->S2', 'REM->REM']
# feats_cant_seperate_both_healthy2 = ['W->REM', 'S1->W', 'S1->S2', 'S2->S1', 'S2->REM', 'S3->S2', 'S3->S4', 'S4->S1', 'S4->S2', 'REM->W', 'REM->S1', 'REM->S2']

theoritically_infeasible_transitions = ['W->S3', 'W->S4', 'W->REM', 'S1->S3', 'S1->S4', 'S1->REM', 'S3->S1', 'S3->REM', 'S4->S1', 'S4->REM', 'REM->S1', 'REM->S3', 'REM->S4'] 

len(AUC_0_7),len(AUC_0_5),len(RF_FI_0_2) 


top_feats = sel_cols 
# top_feats = get_top_featute_names_baseOn_pValue(stat_dataset, pval=0.05)
# top_feats = get_top_featute_names_baseOn_AUC(stat_dataset, auc=0.6)
len(top_feats)

35

In [186]:
# # Feature values greater than or equal to 1% of the transitions
# processed_dataset = dataset.copy()
# dd = ((processed_dataset.mean()/processed_dataset.mean().sum())>=0.01) 
# dd = dd[dd==True]
# type(dd), dd.index.tolist(), dd
# take_feats = dd.index.tolist() 
# take_feats

In [197]:
processed_dataset['Dataset'].unique()

array(['CAP_Sleep'], dtype=object)

In [200]:
# processed_dataset = dataset.copy() if not random_state_value else dataset.copy().sample(frac=1, random_state=random_state_value).reset_index(drop=True) 
processed_dataset = dataset.copy()
all_cols = processed_dataset.columns.values.tolist() 
last_metadata_col_indx = all_cols.index('Subject_Name')+1 

# ### Select some features based on dataset 
# processed_dataset = processed_dataset[~(processed_dataset['Subject_Name'].str.startswith('SC'))] #dataset[~(dataset['Subject_Name'].str.startswith('SC'))]
# processed_dataset = processed_dataset[~(processed_dataset['Dataset']=='Sleep_EDFX')] #dataset[~(dataset['Subject_Name'].str.startswith('SC'))]

### Select specific group of features from % of transitions
# selected_feats = all_cols[:last_metadata_col_indx]+take_feats
selected_feats = all_cols[:last_metadata_col_indx]+top_feats
selected_feats 
processed_dataset = processed_dataset.loc[:, selected_feats]

# ### Remove zero and wake-to-wake and specific group 
# removable_feats = []
# zero_feats = get_features_with_zero_values(processed_dataset)
# zero_feats
# removable_feats.extend(zero_feats)
# removable_feats.extend(['W->W'])
# removable_feats.extend(feats_cant_seperate_both_healthy) 
# removable_feats.extend(theoritically_infeasible_transitions) 
# print(f'Removed: {removable_feats}')
# removable_feats 
# processed_dataset = processed_dataset.drop(removable_feats, axis=1, errors='ignore')

### Remove features with <5% transitions


### Select specific group of features 
# # # processed_dataset = processed_dataset[all_metadata_columns+sorted_PAUC_df]
# # processed_dataset = processed_dataset.drop(removable_feats, axis=1)
# # # select features from manual feature selection list: from previous AUC or RF 
# selected_feats = all_cols[:last_metadata_col_indx]+theoritically_feasible_transitions
# selected_feats = all_cols[:last_metadata_col_indx]+feats_cant_seperate_both_healthy
# selected_feats 
# processed_dataset = processed_dataset.loc[:, selected_feats]

### #Binary/Multi-class healthy vs disorders 
# processed_dataset = processed_dataset[~processed_dataset['Category'].isin(['brux', 'sdb'])]### Brux and sdb is cancelled coz of low number to fit in 5 fold
label_map, processed_dataset = map_category_to_class(processed_dataset.copy(), source_col='Category', class_name='Class', removable_cats=None, multi_class=False)
# label_map, processed_dataset = map_category_to_class(processed_dataset.copy(), source_col='Category', class_name='Class', removable_cats=None, multi_class=True)
# label_map, processed_dataset = map_category_to_class(processed_dataset.copy(), source_col='Category', class_name='Class', removable_cats=['brux', 'sdb', 'narco', 'ins', 'plm'], multi_class=True) 
# label_map, processed_dataset = map_category_to_class(processed_dataset.copy(), source_col='Category', class_name='Class', removable_cats=['brux', 'sdb', 'narco', 'ins', 'plm'], multi_class=True) 
# label_map, processed_dataset = map_category_to_class(processed_dataset.copy(), source_col='Category', class_name='Class', removable_cats=['brux', 'sdb', 'narco', 'ins', 'plm'], multi_class=True) 

# label_map, processed_dataset = map_category_to_class(processed_dataset.copy(), source_col='Category', class_name='Class', removable_cats=None, multi_class=False, age_data=all_demography_detail_df.copy(), age_col = 'Age', source_cols=['Subject_Name', 'File_Name'])
# label_map, processed_dataset = map_category_to_class(processed_dataset.copy(), source_col='Category', class_name='Class', removable_cats=None, multi_class=False, age_data=None, age_col = 'Age', source_cols=['Subject_Name', 'File_Name'])
# label_map, processed_dataset = map_category_to_class(processed_dataset.copy(), source_col='Category', class_name='Class', removable_cats=['brux', 'sdb', 'narco', 'ins', 'plm', 'nfle'], multi_class=False, age_data=None, age_col = 'Age', source_cols=['Subject_Name', 'File_Name'])
# label_map, processed_dataset = map_category_to_class(processed_dataset.copy(), source_col='Category', class_name='Class', removable_cats=['brux', 'sdb', 'narco', 'ins', 'rbd', 'nfle'], multi_class=False, age_data=None, age_col = 'Age', source_cols=['Subject_Name', 'File_Name'])


print(label_map)
# processed_dataset = processed_dataset[~processed_dataset['Subject_Name'].str.startswith('SC')] 
# processed_dataset = processed_dataset[~processed_dataset['Subject_Name'].str.startswith('ST')] 
# processed_dataset = processed_dataset[~(processed_dataset['Dataset']=='Sleep_EDFX')]
# processed_dataset = processed_dataset[~(processed_dataset['Dataset']=='SDRC')]
processed_dataset = processed_dataset[~((processed_dataset['Dataset'].isin(['CAP_Sleep','SDRC'])) & (processed_dataset['Category'].isin(['n','ins'])))]

# ### #Binary/Multi-class age-group detection  
# label_map, processed_dataset = map_age_category_to_class(processed_dataset.copy(), all_demography_detail_df.copy(), source_cols=['Subject_Name', 'File_Name'], age_col ='Age', age_ranges = [[0, 30], [40, 200]], class_name='Class', multi_class=True, healthy_only=True) ##19-101 
# label_map, processed_dataset = map_age_category_to_class(processed_dataset.copy(), all_demography_detail_df.copy(), source_cols=['Subject_Name', 'File_Name'], age_col ='Age', age_ranges = [[0, 30], [40, 60], [70, 200]], class_name='Class', multi_class=True, healthy_only=True) ##19-101 
# label_map, processed_dataset = map_age_category_to_class(processed_dataset.copy(), all_demography_detail_df.copy(), source_cols=['Subject_Name', 'File_Name'], age_col ='Age', age_ranges = [[0, 35], [35, 55], [55, 200]], class_name='Class', multi_class=True, healthy_only=True) ##19-101  
# label_map, processed_dataset = map_age_category_to_class(processed_dataset.copy(), all_demography_detail_df.copy(), source_cols=['Subject_Name', 'File_Name'], age_col ='Age', age_ranges = [[0, 35], [55, 200]], class_name='Class', multi_class=True, healthy_only=True) ##19-101 
# print(label_map)
# processed_dataset

processed_dataset = processed_dataset.reset_index(drop=True) 
# processed_dataset = processed_dataset.reset_index() 
processed_dataset 

['n', 'brux', 'sdb', 'ins', 'narco', 'nfle', 'plm', 'rbd']
['n', 'brux', 'sdb', 'ins', 'narco', 'nfle', 'plm', 'rbd']
{'n': 0, 'brux': 1, 'sdb': 1, 'ins': 1, 'narco': 1, 'nfle': 1, 'plm': 1, 'rbd': 1}


,Dataset,Category,Subject_Name,Class,W->S1,W->S2,W->S3,W->S4,W->REM,S1->W,...,S4->S2,S4->S3,S4->S4,S4->REM,REM->W,REM->S1,REM->S2,REM->S3,REM->S4,REM->REM
0,CAP_Sleep,brux,brux1,1,0.192308,0.009615,0.0,0.0,0.000000,0.030769,...,0.004926,0.024631,0.970443,0.0,0.033520,0.000000,0.000000,0.0,0.0,0.966480
1,CAP_Sleep,brux,brux2,1,0.176000,0.000000,0.0,0.0,0.000000,0.061728,...,0.000000,0.012048,0.984940,0.0,0.019324,0.004831,0.000000,0.0,0.0,0.975845
2,CAP_Sleep,sdb,sdb1,1,0.123853,0.000000,0.0,0.0,0.000000,0.099237,...,0.005076,0.030457,0.949239,0.0,0.009346,0.000000,0.028037,0.0,0.0,0.962617
3,CAP_Sleep,sdb,sdb2,1,0.045161,0.006452,0.0,0.0,0.000000,0.011364,...,0.008475,0.016949,0.966102,0.0,0.000000,0.000000,0.025641,0.0,0.0,0.974359
4,CAP_Sleep,sdb,sdb3,1,0.046948,0.018779,0.0,0.0,0.000000,0.108696,...,0.025974,0.038961,0.935065,0.0,0.062500,0.000000,0.000000,0.0,0.0,0.937500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,Sleep_EDFX,n,ST7191,0,0.246377,0.014493,0.0,0.0,0.000000,0.044118,...,0.013889,0.166667,0.805556,0.0,0.000000,0.020000,0.015000,0.0,0.0,0.965000
179,Sleep_EDFX,n,ST7201,0,0.588235,0.058824,0.0,0.0,0.058824,0.014493,...,0.038462,0.615385,0.346154,0.0,0.044776,0.007463,0.014925,0.0,0.0,0.932836
180,Sleep_EDFX,n,ST7211,0,0.100000,0.000000,0.0,0.0,0.005882,0.116667,...,0.016949,0.254237,0.728814,0.0,0.025641,0.000000,0.019231,0.0,0.0,0.955128
181,Sleep_EDFX,n,ST7221,0,0.181208,0.000000,0.0,0.0,0.000000,0.052133,...,0.000000,0.000000,0.000000,0.0,0.021645,0.043290,0.000000,0.0,0.0,0.935065


In [201]:
# processed_dataset[~processed_dataset['Subject_Name'].str.startswith('SC')].groupby('Category')['Class'].value_counts() 
processed_dataset.mean()
# processed_dataset.isna().sum()
# label_map

Class       0.453552
W->S1       0.101843
W->S2       0.013957
W->S3       0.000739
W->S4       0.000044
W->REM      0.003159
S1->W       0.064089
S1->S1      0.722919
S1->S2      0.194690
S1->S3      0.000816
S1->S4      0.000130
S1->REM     0.017354
S2->W       0.011961
S2->S1      0.015392
S2->S2      0.930085
S2->S3      0.032220
S2->S4      0.000326
S2->REM     0.010017
S3->W       0.007834
S3->S1      0.004172
S3->S2      0.217264
S3->S3      0.658983
S3->S4      0.061760
S3->REM     0.000807
S4->W       0.004381
S4->S1      0.001950
S4->S2      0.008586
S4->S3      0.121993
S4->S4      0.611426
S4->REM     0.000298
REM->W      0.015896
REM->S1     0.016441
REM->S2     0.015615
REM->S3     0.000075
REM->S4     0.000000
REM->REM    0.951972
dtype: float64

In [202]:
print(processed_dataset['Dataset'].unique(), processed_dataset['Class'].unique()) 
processed_dataset.groupby('Category')['Class'].value_counts()
# processed_dataset['Category'].unique().tolist()

['CAP_Sleep' 'Sleep_EDFX'] [1 0]


Category  Class
brux      1          2
n         0        100
narco     1          5
nfle      1         40
plm       1         10
rbd       1         22
sdb       1          4
Name: Class, dtype: int64

In [203]:
if logger:
    stop_logger(logger) 
util, logger = start_logger(result_save_path, exp_name)

# classifier_obj = HumachLab_ML_CLassifiers(logger=logger, directory=result_save_path, dataset=dataset.copy(), class_name=class_name, metadata_column=metadata_column, split_column=split_column) 
classifier_obj = HumachLab_ML_CLassifiers(logger=logger, directory=result_save_path, dataset=processed_dataset.copy(), class_name=class_name, label_map=label_map, metadata_column=metadata_column, split_column=split_column, random_state_value=random_state_value, split_balance_pattern=split_balance_pattern) 

classifier_obj 


NameError: name 'HumachLab_ML_CLassifiers' is not defined

In [176]:
classifier_obj.print_message()

Hello from HumachLab_ML_CLassifiers class


In [43]:
# splitting_crieteria = [(10, 0), (5, 20)]     ### for test & training (validation) splitting_crieteria (m, n)-m folds, n%:  m=0: loso, m>0: m-fold, (n>0 given m>0) -shuffled random splitting with m-fold, n% testing 
splitting_crieteria = [(5, 0), (5, 20)]     ### for test & training (validation) splitting_crieteria (m, n)-m folds, n%:  m=0: loso, m>0: m-fold, (n>0 given m>0) -shuffled random splitting with m-fold, n% testing 
# splitting_crieteria = [(1, 0), (1, 20)]   ## for small data  ### for test & training (validation) splitting_crieteria (m, n)-m folds, n%:  m=0: loso, m>0: m-fold, (n>0 given m>0) -shuffled random splitting with m-fold, n% testing 
# splitting_crieteria = [(5, 0), (2, 20)]   ## for small data  ### for test & training (validation) splitting_crieteria (m, n)-m folds, n%:  m=0: loso, m>0: m-fold, (n>0 given m>0) -shuffled random splitting with m-fold, n% testing 
model_list = [ML_Classifiers.LogReg, ML_Classifiers.SVC, ML_Classifiers.NB, ML_Classifiers.kNN, ML_Classifiers.DT, ML_Classifiers.RF, ML_Classifiers.GBoost, ML_Classifiers.XGBoost] # [ML_Classifiers.LogReg, ML_Classifiers.SVC, ML_Classifiers.NB, ML_Classifiers.kNN, ML_Classifiers.DT, ML_Classifiers.RF, ML_Classifiers.GBoost] 
should_use_params = True 
is_validate_models = True
# is_binary_classification = False 
apply_feature_selection = False
custom_splitter = True
exp_name = exp_name

splitting_crieteria, model_list, result_save_path, should_use_params 

([(5, 0), (5, 20)],
 [<ML_Classifiers.LogReg: 'logistic_regression'>,
  <ML_Classifiers.SVC: 'support_vector_classifier'>,
  <ML_Classifiers.NB: 'naive_bayes'>,
  <ML_Classifiers.kNN: 'k_nearest_neighbors'>,
  <ML_Classifiers.DT: 'decision_tree'>,
  <ML_Classifiers.RF: 'random_forest'>,
  <ML_Classifiers.GBoost: 'gradient_boosting'>,
  <ML_Classifiers.XGBoost: 'xtreme_gradient_boosting'>],
 './Results//_Classification/ML10001/',
 True)

In [44]:
classifier_obj.class_name, classifier_obj.split_column, result_directory

NameError: name 'classifier_obj' is not defined

In [100]:
### Set the classifier parameters in the "HumachLab_ML_CLassifiers" class file to run with the parameter 
# best_tr_model, tr_model, tr_model_scores_df, tr_target_and_prediction_df, ts_model, ts_model_scores_df, ts_target_and_prediction_df, ts_fold_info_df, exp_info_df

stop_logger(logger) 

# exp_sum_df = modify_experiment_information_summarry(result_directory, dict_dat=exp_detail) 
# exp_sum_df 

AttributeError: 'NoneType' object has no attribute 'handlers'

In [204]:
model_save_path = f"{result_save_path}Models/" 
model_save_path

'./Results//_Classification/ML10008/Models/'

In [205]:
import glob 
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier 


class HumachLab_ML_Predictors:
    def __init__(self, logger, result_save_path, dataset, class_name, label_map, metadata_column):
        self.logger = logger 
        self.result_save_path = result_save_path 
        self.dataset = dataset  
        self.class_name = class_name  
        self.label_map = label_map 
        self.metadata_column = metadata_column 
        
        
    def get_trained_models(self): 
        self.model_save_path = f"{self.result_save_path}Models/"  
        
        all_model_files = glob.glob(f"{self.model_save_path}best*") 
#         print(all_model_files)
        
        save_path = f"{self.model_save_path}best_tr_model" 
        all_best_tr_model = self.load_models_from_file(save_path, 'Best Training Models')        
        
        return all_model_files, all_best_tr_model     

    
    def load_models_from_file(self, save_path, model_type):
        models_dict = {} 
        
        save_path = f'{save_path}*'
        files = self.sort_string_list(glob.glob(save_path)) 
        files
        selected_files = [[int(fn) for fn in f[len(save_path):].split('.')[0].split('_')] for f in files]
        selected_files
        
        self.logger.info(f'Start retrieving {model_type} model from file...')
        model_dict = {}  
        for i, (ind, fl) in enumerate(zip(selected_files, files)):
            if len(ind)==3:
                tsi, tri, modi = ind
                mod = None 
                with open(fl, "rb") as f:
                    mod = pickle.load(f) 
                if tsi not in model_dict.keys(): 
                    model_dict[tsi] = {}
                if tri not in model_dict[tsi].keys(): 
                    model_dict[tsi][tri] = {} 
                if modi not in model_dict[tsi][tri].keys(): 
                    model_dict[tsi][tri][modi] = mod 
            elif len(ind)==2:
                tsi, modi = ind
                mod = None 
                with open(fl, "rb") as f:
                    mod = pickle.load(f) 
                if tsi not in model_dict.keys(): 
                    model_dict[tsi] = {}
                if modi not in model_dict[tsi].keys(): 
                    model_dict[tsi][modi] = mod 
            else:
                self.logger.info(f'Doesn\'t identify {model_type} model file to retrieve...')
        
        model_dict
        self.logger.info(f'Finish retrieving {model_type} model from file...')
        return model_dict     
    
    
        
    def save_results(self, save_directory, all_pred_scores_df):
        # './Results/_Classification/ML001/'
        save_path = f"{save_directory}all_pred_scores.csv" 
        all_pred_scores_df.to_csv(save_path, index=False) 
        print(f"Saving prediction result to: {save_path}") 
        
        return
    
    
    def sort_string_list(self, string_list):
        ## ref: https://blog.codinghorror.com/sorting-for-humans-natural-sort-order/
        """ Sort the given list in the way that humans expect.
        """
        convert = lambda text: int(text) if text.isdigit() else text
        alphanum_key = lambda key: [ convert(c.replace("_","")) for c in re.split('([0-9]+)', key) ]
        string_list.sort( key=alphanum_key )
        return string_list
    
    
    def prediction(self, model_list, all_best_tr_model):
        all_pred_score_df = self.do_prediction(model_list, all_best_tr_model) 
        self.save_results(self.result_save_path, all_pred_score_df)  
        return all_pred_score_df 
    
    
    def do_prediction(self, model_list, all_best_tr_model):
        all_pred_score_df = pd.DataFrame() 
        all_cols = self.dataset.columns.values.tolist() 
        feat_names = all_cols[all_cols.index(self.class_name)+1:] 
        all_feats = self.dataset[feat_names] 
        labels = self.dataset[self.class_name].values.tolist()  
        mod_act_name_list = [ML_Classifiers.get_actual_name(str(tmp_name._name_)) for tmp_name in model_list] 

#         model_ids = range(1, len(model_list)+1) #[1] #range(1, len(model_list)+1)

        for iter_key, iter_val in all_best_tr_model.items(): 
            for model_key, model_val in iter_val.items(): 
#                 print('###===>', iter_key, model_key) 
                print('###===>', iter_key, model_key, model_val.best_estimator_.__class__.__name__)
                mod_feats = model_val.feature_names 
        #         if 'index' in mod_feats:
        #             mod_feats.remove('index')
                feat_names.sort()
                mod_feats.sort()
#                 print('===>', model_key, model_val, model_list[model_key-1], mod_act_name_list[model_key-1]) 
                mod_act_name = model_val.best_estimator_.__class__.__name__                 
                
#                 ML_Classifiers.get_actual_name(str(ML_Classifiers.LogReg._name_)) 
                if (feat_names == mod_feats) and (mod_act_name in mod_act_name_list):
                    print('###===>', iter_key, model_key, model_val.best_estimator_.__class__.__name__, mod_act_name_list)
#                     print('***===>', iter_key, model_key, model_val, model_val.__dict__)
                    prediction = model_val.predict(all_feats) 
#                     prediction = prediction.tolist() 
                    prediction_proba = model_val.predict_proba(all_feats) 
#                     prediction_proba = prediction_proba.tolist() 
                    
                    scores_df = self.calculate_model_scores(model_val, labels, prediction, prediction_proba)
                    scores_df.insert(0, "Model_No", model_key) 
                    scores_df.insert(0, "Test_No", iter_key) 
                    all_pred_score_df = pd.concat([all_pred_score_df, scores_df]) 
                else:
                    if (feat_names == mod_feats): 
                        print('Model is not found.')  
                    else:
                        print('Data feature and model features are not same: \n', feat_names, '\n', mod_feats) 
        if len(all_pred_score_df)>0:
            all_pred_score_df = all_pred_score_df.sort_values(by=["Model_No", "Test_No"])
        return all_pred_score_df
    

    
    # #########################################################################
    # Calculate and save classification details and model scores
    # #########################################################################
    #############################

    def calculate_model_scores(self, mods, y_test, y_pred, y_pred_proba): 
#         print(y_test, '\n', y_pred, '\n', y_pred_proba, '\n')
#         target_labels = np.unique(np.array(y_test)).tolist() 
        target_labels = sorted( self.dataset[self.class_name].unique().tolist() )
        
        y_pred = y_pred.tolist() 
        perf_scores = self.calculate_performance_scores(y_test, y_pred, y_pred_proba, labels=target_labels)  # average = 'weighted', 'macro', 'micro' 
        confMat = perf_scores['Conf_Mat']

        acc = round(perf_scores['ACC'], 3) #Accuracy score or total correct prediction rate 
        prec = round(perf_scores['PREC'], 3) #precision or positive predictive value (PPV)
        reca_sens = round(perf_scores['REC'], 3) #sensitivity, recall, hit rate, or true positive rate (TPR)
        spec = round(perf_scores['SPE'], 3) #specificity, selectivity or true negative rate (TNR)
        f1sc = round(perf_scores['F1SCR'], 3)
        auc_s = round(perf_scores['AUC'], 3) 
        
        scr_dict = {'method': str(mods), 'model': mods.best_estimator_, 'model_parameters': mods.best_params_, 
                    'model_scores': round(mods.best_score_*100,2),
                    ML_Performace_Metrics.CONF_MAT.value: confMat, ML_Performace_Metrics.ACC.value: acc, ML_Performace_Metrics.PREC.value: prec,
                    ML_Performace_Metrics.RECL.value: reca_sens, ML_Performace_Metrics.SEN.value: reca_sens, ML_Performace_Metrics.SPEC.value: spec,
                    ML_Performace_Metrics.F1SCR.value: f1sc, ML_Performace_Metrics.ROC_AUC.value: auc_s}
        
#         scr_dict = {'method_class': str(mods.__class__.__name__), 'model_name': mods.best_estimator_, 'model_parameters': mods.best_params_, 
#                     'model_scores': round(mods.best_score_*100,2),
#                     ML_Performace_Metrics.CONF_MAT.value: confMat, ML_Performace_Metrics.ACC.value: acc, ML_Performace_Metrics.PREC.value: prec,
#                     ML_Performace_Metrics.RECL.value: reca_sens, ML_Performace_Metrics.SEN.value: reca_sens, ML_Performace_Metrics.SPEC.value: spec,
#                     ML_Performace_Metrics.F1SCR.value: f1sc, ML_Performace_Metrics.ROC_AUC.value: auc_s}
#         scr_dict = {'model_class': str(mods.__class__.__name__), 'method_name': mods.estimator, 'method_parameters': mods.best_params_, 
#                     'method_scores': round(mods.best_score_*100,2),
#                     ML_Performace_Metrics.CONF_MAT.value: confMat, ML_Performace_Metrics.ACC.value: acc, ML_Performace_Metrics.PREC.value: prec,
#                     ML_Performace_Metrics.RECL.value: reca_sens, ML_Performace_Metrics.SEN.value: reca_sens, ML_Performace_Metrics.SPEC.value: spec,
#                     ML_Performace_Metrics.FPR.value: fpr, ML_Performace_Metrics.FNR.value: fnr, ML_Performace_Metrics.F1SCR.value: f1sc, ML_Performace_Metrics.ROC_AUC.value: auc_s}
#         scr_dict = {'model_class': str(mods.__class__.__name__), 'method_name': str(mods), 'method_parameters': mods.best_params_, 
#                     'method_scores': round(mods.best_score_*100,2),
#                     ML_Performace_Metrics.CONF_MAT.value: confMat, ML_Performace_Metrics.ACC.value: acc, ML_Performace_Metrics.PREC.value: prec,
#                     ML_Performace_Metrics.RECL.value: reca_sens, ML_Performace_Metrics.SEN.value: reca_sens, ML_Performace_Metrics.SPEC.value: spec,
#                     ML_Performace_Metrics.FPR.value: fpr, ML_Performace_Metrics.FNR.value: fnr, ML_Performace_Metrics.F1SCR.value: f1sc, ML_Performace_Metrics.ROC_AUC.value: auc_s}
        scr_df = pd.DataFrame([list( scr_dict.values() )], columns=list( scr_dict.keys() )) 
        self.logger.info(f"""Score columns: {scr_df.shape} {scr_df.columns.values.tolist()}""") 

        return scr_df
    
    
    def calculate_performance_scores(self, y_true, y_pred, y_pred_proba, labels=[0, 1], verbose=2, average='weighted'): # average = 'macro', 'micro', 'weighted' 
        #### SOURCES: https://www.youtube.com/watch?v=PCHf_7jBor8 
        # https://www.mariakhalusova.com/posts/2019-04-17-ml-model-evaluation-metrics-p2/ 
        # https://www.mariakhalusova.com/posts/2019-04-11-ml-model-evaluation-metrics-p1/
        # https://www.evidentlyai.com/classification-metrics/multi-class-metrics 
        # https://www.kaggle.com/code/nkitgupta/evaluation-metrics-for-multi-class-classification 

        model_scores = []
        true_label_uniq = np.unique(np.array(y_true)).tolist() 
        print(np.unique(np.array(y_true)), np.unique(np.array(y_pred)))
        print(y_true, y_pred) 
        conf_matrix = confusion_matrix(y_true, y_pred, labels=labels).tolist()
        print(np.array(conf_matrix) )

        ### For micro averaging and binary class 
        conf_matrix_arr = np.array(conf_matrix) 
        one_vs_all_confMat = []     
        for label in labels:
            tp_lbl = conf_matrix_arr[label, label] 
            fp_lbl = np.sum(conf_matrix_arr[:, label])-tp_lbl 
            fn_lbl = np.sum(conf_matrix_arr[label, :])-tp_lbl 
            tn_lbl = np.sum(conf_matrix_arr)-(tp_lbl+fp_lbl+fn_lbl) 
            one_vs_all_confMat.append([tn_lbl, fp_lbl, fn_lbl, tp_lbl]) 
        print(np.array(one_vs_all_confMat)) 

        tn_tot = np.sum( np.array(one_vs_all_confMat)[:, 0] ) 
        fp_tot = np.sum( np.array(one_vs_all_confMat)[:, 1] )  
        fn_tot = np.sum( np.array(one_vs_all_confMat)[:, 2] )  
        tp_tot = np.sum( np.array(one_vs_all_confMat)[:, 3] )

        conf_matrix_tol = [[tn_tot, fp_tot], [fn_tot, tp_tot]] 
        print(np.array(conf_matrix_tol)) 

        if len(labels)==2:
            tn_tot = one_vs_all_confMat[1][0] 
            fp_tot = one_vs_all_confMat[1][1] 
            fn_tot = one_vs_all_confMat[1][2] 
            tp_tot = one_vs_all_confMat[1][3] 
            average = "micro"

        result = [] 
        for label in labels:
            precision, recall, fscore, support = precision_recall_fscore_support( np.array(y_true)==label, np.array(y_pred)==label ) 
            # tmp_fpr, tmp_tpr, tmp_thresholds = roc_curve(np.array(y_true)==label, np.array(y_pred)==label, pos_label=label) 
            tmp_fpr, tmp_tpr, tmp_thresholds = roc_curve(np.array(y_true)==label, np.array(y_pred)==label) 
            auc_score = auc(tmp_fpr, tmp_tpr)*100 

            if label in true_label_uniq: 
                result.append( [label, precision[1], recall[1], recall[1], recall[0], fscore[1], auc_score, support[1]] ) 
            else:
                result.append( [label, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0] ) 

            accuracy = accuracy_score(np.array(y_true)==label, np.array(y_pred)==label)*100 
            if verbose>1:
                print(
                    f'Class-wise info: For multilevel internal scores fo label {label}: \n', 
                    f'Accuracy = {accuracy}\n', 
                    f'Precision = {precision}\n', 
                    f'Recall = {recall}\n', 
                    f'F1 score = {fscore}\n', 
                    f'AUC score = {auc_score}\n', 
                    f'Support = {support}\n', 
                )
        tdf = pd.DataFrame(result, columns=['Label', 'Precision', 'Recall', 'Sensitivity', 'Specificity', 'F1 Score', 'AUC', 'Support']) 

        if average=='macro': #average = "weighted", "macro", "micro" 
            tdf = tdf[['Precision', 'Recall', 'Sensitivity', 'Specificity', 'F1 Score', 'AUC']].apply(lambda col:np.mean(col), axis=0) 
        elif average=='micro':
            prc = (tp_tot / (tp_tot+fp_tot))*100 if (tp_tot+fp_tot)!=0 else 0.0 #precision or positive predictive value (PPV)
            rec = (tp_tot / (tp_tot+fn_tot))*100 if (tp_tot+fn_tot)!=0 else 0.0 #sensitivity, recall, hit rate, or true positive rate (TPR)
            sns = rec #sensitivity same as recall 
            spc = (tn_tot / (tn_tot+fp_tot))*100 if (tn_tot+fp_tot)!=0 else 0.0 #specificity, selectivity or true negative rate (TNR)
            f1s = (2*tp_tot / (2*tp_tot+fp_tot+fn_tot))*100 if (2*tp_tot+fp_tot+fn_tot)!=0 else 0.0 #specificity, selectivity or true negative rate (TNR)  
            auc_s = roc_auc_score(y_true, y_pred) if len(labels)==2 else roc_auc_score(y_true, y_pred_proba, multi_class='ovr', average=average) #multi_class='ovo', 'ovr' 
#             auc_s = 0.5
#             if len(labels)==2:
#                 auc_s = roc_auc_score(y_true, y_pred)  
#             else:
#                 auc_s = roc_auc_score(y_true, y_pred_proba, multi_class='ovr', average=average) #multi_class='ovo', 'ovr' 
            tdf = pd.Series([prc, rec, sns, spc, f1s, auc_s], index=['Precision', 'Recall', 'Sensitivity', 'Specificity', 'F1 Score', 'AUC'])  
        else: ## Default = weighted
            class_weights = tdf['Support']/tdf['Support'].sum() 
            tdf = tdf[['Precision', 'Recall', 'Sensitivity', 'Specificity', 'F1 Score', 'AUC']].apply(lambda col:np.sum(col*class_weights), axis=0) 

        acc = accuracy_score(y_true, y_pred)*100 
        tdf_summary = pd.Series([conf_matrix, acc, tdf['Precision'], tdf['Recall'], tdf['Sensitivity'], tdf['Specificity'], tdf['F1 Score'], tdf['AUC']],
                               index=['Conf_Mat', 'ACC', 'PREC', 'REC', 'SEN', 'SPE', 'F1SCR', 'AUC'])

        if verbose>1:
            confMat = tdf_summary['Conf_Mat']
            acc = round(tdf_summary['ACC'], 3) #Accuracy score or total correct prediction rate 
            prec = round(tdf_summary['PREC'], 3) #precision or positive predictive value (PPV)
            reca_sens = round(tdf_summary['REC'], 3) #sensitivity, recall, hit rate, or true positive rate (TPR)
            spec = round(tdf_summary['SPE'], 3) #specificity, selectivity or true negative rate (TNR)
            f1sc = round(tdf_summary['F1SCR'], 3)
            auc_s = round(tdf_summary['AUC'], 3) 
            print(
                f'CLASSIFICATION MERICS:\n',
                f'{"_"*55}\n',
                f'Confusion Matrix: \n{np.array(conf_matrix)}\n',
                f'Accuracy (acc): {acc}\n',
                f'Precision (prc): {prec}\n',
                f'Recall (rec): {reca_sens}\n',
                f'Sensitivity (sns): {reca_sens}\n',
                f'Specificity (spc): {spec}\n',
                f'F1 Score (f1s): {f1sc}\n',
                f'ROC AUC (AUC): {auc_s}',
            )

        return tdf_summary
    
    

In [206]:
if logger:
    stop_logger(logger) 
util, logger = start_logger(result_save_path, exp_name)

predictor_obj = HumachLab_ML_Predictors(logger=logger, result_save_path=result_save_path, dataset=processed_dataset.copy(), class_name=class_name, label_map=label_map, metadata_column=metadata_column) 

predictor_obj 

In [207]:
all_model_files, all_best_tr_model = predictor_obj.get_trained_models()

Start retrieving Best Training Models model from file...
Finish retrieving Best Training Models model from file...


In [208]:
# len(all_model_files), all_best_tr_model 

In [209]:
predictor_obj.prediction(model_list, all_best_tr_model) 

###===> 1 1 LogisticRegression
###===> 1 1 LogisticRegression ['LogisticRegression', 'SVC', 'GaussianNB', 'KNeighborsClassifier', 'DecisionTreeClassifier', 'RandomForestClassifier', 'GradientBoostingClassifier', 'XGBClassifier']
[0 1] [0 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

C:\Users\aliem\.conda\envs\Python37Work\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
C:\Users\aliem\.conda\envs\Python37Work\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
C:\Users\aliem\.conda\envs\Python37Work\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but SVC was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
C:\Users\aliem\.conda\envs\Python37Work\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but SVC was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
C:\Users\aliem\.conda\envs\Python37Work\lib\site-packa

CLASSIFICATION MERICS:
 _______________________________________________________
 Confusion Matrix: 
[[100   0]
 [ 83   0]]
 Accuracy (acc): 54.645
 Precision (prc): 0.0
 Recall (rec): 0.0
 Sensitivity (sns): 0.0
 Specificity (spc): 100.0
 F1 Score (f1s): 0.0
 ROC AUC (AUC): 0.5
Score columns: (1, 12) ['method', 'model', 'model_parameters', 'model_scores', 'confusion_matrix', 'accuracy', 'precision', 'recall', 'sensitivity', 'specificity', 'f1_score', 'roc_auc']
###===> 1 5 DecisionTreeClassifier
###===> 1 5 DecisionTreeClassifier ['LogisticRegression', 'SVC', 'GaussianNB', 'KNeighborsClassifier', 'DecisionTreeClassifier', 'RandomForestClassifier', 'GradientBoostingClassifier', 'XGBClassifier']
[0 1] [0 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

C:\Users\aliem\.conda\envs\Python37Work\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
C:\Users\aliem\.conda\envs\Python37Work\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
C:\Users\aliem\.conda\envs\Python37Work\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
C:\Users\aliem\.conda\envs\Python37Work\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
C:\Users

[0 1] [0 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

...

C:\Users\aliem\.conda\envs\Python37Work\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
C:\Users\aliem\.conda\envs\Python37Work\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
C:\Users\aliem\.conda\envs\Python37Work\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but SVC was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
C:\Users\aliem\.conda\envs\Python37Work\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but SVC was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
C:\Users\aliem\.conda\envs\Python37Work\lib\site-packa

###===> 5 6 RandomForestClassifier
###===> 5 6 RandomForestClassifier ['LogisticRegression', 'SVC', 'GaussianNB', 'KNeighborsClassifier', 'DecisionTreeClassifier', 'RandomForestClassifier', 'GradientBoostingClassifier', 'XGBClassifier']
[0 1] [0 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,

C:\Users\aliem\.conda\envs\Python37Work\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"
C:\Users\aliem\.conda\envs\Python37Work\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


[[19 81]
 [39 44]]
[[44 39 81 19]
 [19 81 39 44]]
[[ 63 120]
 [120  63]]
Class-wise info: For multilevel internal scores fo label 0: 
 Accuracy = 34.42622950819672
 Precision = [0.352      0.32758621]
 Recall = [0.53012048 0.19      ]
 F1 score = [0.42307692 0.24050633]
 AUC score = 36.00602409638554
 Support = [ 83 100]

Class-wise info: For multilevel internal scores fo label 1: 
 Accuracy = 34.42622950819672
 Precision = [0.32758621 0.352     ]
 Recall = [0.19       0.53012048]
 F1 score = [0.24050633 0.42307692]
 AUC score = 36.006024096385545
 Support = [100  83]

CLASSIFICATION MERICS:
 _______________________________________________________
 Confusion Matrix: 
[[19 81]
 [39 44]]
 Accuracy (acc): 34.426
 Precision (prc): 35.2
 Recall (rec): 53.012
 Sensitivity (sns): 53.012
 Specificity (spc): 19.0
 F1 Score (f1s): 42.308
 ROC AUC (AUC): 0.36
Score columns: (1, 12) ['method', 'model', 'model_parameters', 'model_scores', 'confusion_matrix', 'accuracy', 'precision', 'recall', 'sens

,Test_No,Model_No,method,model,model_parameters,model_scores,confusion_matrix,accuracy,precision,recall,sensitivity,specificity,f1_score,roc_auc
0,1,1,"GridSearchCV(cv=5, estimator=LogisticRegressio...","LogisticRegression(C=0.5, max_iter=50)","{'C': 0.5, 'max_iter': 50, 'penalty': 'l2'}",26.67,"[[78, 22], [83, 0]]",42.623,0.000,0.000,0.000,78.0,0.000,0.390
0,2,1,"GridSearchCV(cv=5, estimator=LogisticRegressio...",LogisticRegression(max_iter=50),"{'C': 1.0, 'max_iter': 50, 'penalty': 'l2'}",46.67,"[[55, 45], [83, 0]]",30.055,0.000,0.000,0.000,55.0,0.000,0.275
0,3,1,"GridSearchCV(cv=5, estimator=LogisticRegressio...","LogisticRegression(C=0.5, max_iter=50)","{'C': 0.5, 'max_iter': 50, 'penalty': 'l2'}",13.33,"[[81, 19], [83, 0]]",44.262,0.000,0.000,0.000,81.0,0.000,0.405
0,4,1,"GridSearchCV(cv=5, estimator=LogisticRegressio...","LogisticRegression(C=0.5, max_iter=50)","{'C': 0.5, 'max_iter': 50, 'penalty': 'l2'}",33.33,"[[51, 49], [83, 0]]",27.869,0.000,0.000,0.000,51.0,0.000,0.255
0,5,1,"GridSearchCV(cv=5, estimator=LogisticRegressio...","LogisticRegression(C=0.5, max_iter=50)","{'C': 0.5, 'max_iter': 50, 'penalty': 'l2'}",40.00,"[[47, 53], [83, 0]]",25.683,0.000,0.000,0.000,47.0,0.000,0.235
0,1,2,"GridSearchCV(cv=5, estimator=SVC(), n_jobs=50,...","SVC(C=0.5, kernel='poly', probability=True)","{'C': 0.5, 'kernel': 'poly', 'probability': True}",53.33,"[[41, 59], [83, 0]]",22.404,0.000,0.000,0.000,41.0,0.000,0.205
0,2,2,"GridSearchCV(cv=5, estimator=SVC(), n_jobs=50,...","SVC(C=0.5, kernel='poly', probability=True)","{'C': 0.5, 'kernel': 'poly', 'probability': True}",73.33,"[[32, 68], [82, 1]]",18.033,1.449,1.205,1.205,32.0,1.316,0.166
0,3,2,"GridSearchCV(cv=5, estimator=SVC(), n_jobs=50,...","SVC(C=0.5, kernel='poly', probability=True)","{'C': 0.5, 'kernel': 'poly', 'probability': True}",33.33,"[[76, 24], [82, 1]]",42.077,4.000,1.205,1.205,76.0,1.852,0.386
0,4,2,"GridSearchCV(cv=5, estimator=SVC(), n_jobs=50,...","SVC(kernel='poly', probability=True)","{'C': 1.0, 'kernel': 'poly', 'probability': True}",46.67,"[[31, 69], [82, 1]]",17.486,1.429,1.205,1.205,31.0,1.307,0.161
0,5,2,"GridSearchCV(cv=5, estimator=SVC(), n_jobs=50,...","SVC(C=0.5, kernel='poly', probability=True)","{'C': 0.5, 'kernel': 'poly', 'probability': True}",58.33,"[[29, 71], [82, 1]]",16.393,1.389,1.205,1.205,29.0,1.290,0.151


In [49]:
all_cols = processed_dataset.columns.values.tolist() 
feat_names = all_cols[all_cols.index(class_name)+1:] 
all_feats = processed_dataset[feat_names] 
print(all_feats.columns.values.tolist()) 
[(i+1, m) for i, m in enumerate(model_list)] 

['W->S1', 'W->S2', 'W->REM', 'S1->W', 'S1->S1', 'S1->S2', 'S1->REM', 'S2->S3', 'S3->S2', 'S3->S3', 'S3->S4', 'S4->S3', 'S4->S4', 'REM->S1']


[(1, <ML_Classifiers.LogReg: 'logistic_regression'>),
 (2, <ML_Classifiers.SVC: 'support_vector_classifier'>),
 (3, <ML_Classifiers.NB: 'naive_bayes'>),
 (4, <ML_Classifiers.kNN: 'k_nearest_neighbors'>),
 (5, <ML_Classifiers.DT: 'decision_tree'>),
 (6, <ML_Classifiers.RF: 'random_forest'>),
 (7, <ML_Classifiers.GBoost: 'gradient_boosting'>),
 (8, <ML_Classifiers.XGBoost: 'xtreme_gradient_boosting'>)]

In [50]:
actual_name_list = [] 
name_list = [] 
for i in range(len(model_list)): 
    print(all_best_tr_model[1][i+1].best_estimator_.__class__.__name__, '==',  model_list[i]._name_, '||',  model_list[i]._value_) 
    actual_name_list.append(all_best_tr_model[1][i+1].best_estimator_.__class__.__name__) 
    name_list.append(model_list[i]._name_) 
    
print(actual_name_list, name_list) 
print(all_best_tr_model[1][i+1].best_estimator_.__class__.__name__, '==',  model_list[i].__dict__) 

LogisticRegression == LogReg || logistic_regression
SVC == SVC || support_vector_classifier
GaussianNB == NB || naive_bayes
KNeighborsClassifier == kNN || k_nearest_neighbors
DecisionTreeClassifier == DT || decision_tree
RandomForestClassifier == RF || random_forest
GradientBoostingClassifier == GBoost || gradient_boosting
XGBClassifier == XGBoost || xtreme_gradient_boosting
['LogisticRegression', 'SVC', 'GaussianNB', 'KNeighborsClassifier', 'DecisionTreeClassifier', 'RandomForestClassifier', 'GradientBoostingClassifier', 'XGBClassifier'] ['LogReg', 'SVC', 'NB', 'kNN', 'DT', 'RF', 'GBoost', 'XGBoost']
XGBClassifier == {'_value_': 'xtreme_gradient_boosting', '_name_': 'XGBoost', '__objclass__': <enum 'ML_Classifiers'>}


In [75]:
ML_Classifiers.get_actual_name(str(ML_Classifiers.LogReg._name_)) 
# ML_Classifiers.get_short_form(str(ML_Classifiers.LogReg._value_)) 

all_best_tr_model

{1: {1: GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=50,
               param_grid={'C': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0],
                           'max_iter': [50, 100, 130, 150, 170, 200],
                           'penalty': ['l2', 'elasticnet']},
               refit='recall', return_train_score=True, scoring=['recall', 'f1'],
               verbose=2),
  2: GridSearchCV(cv=5, estimator=SVC(), n_jobs=50,
               param_grid={'C': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0],
                           'kernel': ['linear', 'rbf', 'poly'],
                           'probability': [True]},
               refit='recall', return_train_score=True, scoring=['recall', 'f1'],
               verbose=2),
  3: GridSearchCV(cv=5, estimator=GaussianNB(), n_jobs=50,
               param_grid={'var_smoothing': [1.0, 0.005623413251903491,
                                             3.1622776601683795e-05,
                                             1.7782794100389227e

### Test saved results

In [ ]:
# exp_name = 'ML3002' 
# result_save_path = f'./Results/_Classification/{exp_name}/'
# result_save_path

In [ ]:
# if logger:
#     stop_logger(logger) 
# util, logger = start_logger(result_save_path, exp_name)


In [ ]:
# result_save_path2 = result_save_path 
result_save_path2 = f'./Results/_Classification/{exp_name}/'
result_save_path2

In [ ]:
class_name

In [ ]:
if logger:
    stop_logger(logger) 
util, logger = start_logger('./Results/', '00')

classifier_obj2 = HumachLab_ML_CLassifiers(logger=logger, directory=result_save_path2, dataset=pd.DataFrame(columns=[class_name]), class_name=class_name, label_map={}, metadata_column=metadata_column, split_column=split_column, random_state_value=0, split_balance_pattern=[], check_result=True) 

classifier_obj2 

In [ ]:
best_tr_model2, tr_model2, tr_model_scores_df2, tr_target_and_prediction_df2, ts_model2, ts_model_scores_df2, ts_target_and_prediction_df2, ts_fold_info_df2, exp_info_df2  = classifier_obj2.load_results(result_save_path2)  


In [ ]:
# best_tr_model, tr_model, tr_model_scores_df, tr_target_and_prediction_df, ts_model, ts_model_scores_df, ts_target_and_prediction_df, ts_fold_info_df, exp_info_df = best_tr_model2, tr_model2, tr_model_scores_df2, tr_target_and_prediction_df2, ts_model2, ts_model_scores_df2, ts_target_and_prediction_df2, ts_fold_info_df2, exp_info_df2 


In [ ]:
ts_fold_info_df2

In [ ]:
tt = ts_fold_info_df2[ ts_fold_info_df2['Model_No']==1 ] ['Selected_Features'] 
tt

for t in tt:
    print(t) 

In [ ]:
ts_model_scores_df2

##### Confusion Matrix 

In [722]:
# ts_model_scores_df = ts_model_scores_df2

In [ ]:
# label_map = {'n': 0, 'ins': 1, 'narco': 2, 'nfle': 3, 'plm': 4, 'rbd': 5}

label_map

In [ ]:
np.unique(np.array(list(label_map.values())))

In [ ]:
label_map

label_map = {'n': 0, 'dis': 1}

label_map 

In [ ]:
ts_model_scores_df[(ts_model_scores_df['Model_No']==6)] 

In [ ]:
all_conf_mat = ts_model_scores_df[(ts_model_scores_df['Model_No']==6)]['confusion_matrix'].values 
# all_conf_mat = [eval(cf) for cf in all_conf_mat] 
all_conf_mat 

In [ ]:
len(all_conf_mat[0]), len(all_conf_mat[0][0])

In [ ]:
final_conf_mat = np.zeros( ( len(all_conf_mat[0]), len(all_conf_mat[0][0]) ) ) 
for i, arr in enumerate(all_conf_mat):
    arr = np.array(arr) 
    if i==0:
        final_conf_mat = np.zeros( (arr.shape[0], arr.shape[1]) ) 
#     print(type(arr), arr)
    final_conf_mat += arr
    
# tot_counts = np.array( np.sum(final_conf_mat, axis=1) ) 
tot_counts = np.sum(final_conf_mat, axis=1).reshape(1,final_conf_mat.shape[1]) 
print(type(tot_counts), tot_counts) 
final_conf_mat 

In [ ]:
tmp_conf = np.concatenate( (tot_counts, final_conf_mat), axis=0).astype(int) 
tmp_conf

In [ ]:
tmp_conf_df = pd.DataFrame(tmp_conf, columns=list(label_map.keys()), index=['Total']+list(label_map.keys())) 
tmp_conf_df 

In [ ]:
result_save_path, f'{result_save_path}combined_conf_mat.csv'

In [ ]:
tmp_conf_df.to_csv(f'{result_save_path}combined_conf_mat.csv')  
# tmp_conf_df

In [ ]:
tmp_conf_df = pd.read_csv(f'{result_save_path}combined_conf_mat.csv', index_col='Unnamed: 0') 
tmp_conf_df 

# ML Classifier class

In [157]:
### Data preprocessor class 
from scipy.stats import ttest_ind, f_oneway
from sklearn.preprocessing import LabelEncoder

class DataPreprocessor():
    
    def __init__(self):
        return
    
    def calculate_p_and_auc_for_feature(self, feat_data, label_data, binary_class=True): 
        # Extract the independent variable and dependent variable as dataframe and series 
        X = feat_data.copy()  # Replace 'independent_variable' with your column name
        y = label_data.copy()  # Replace 'dependent_variable' with your column name
        # print(X, y) 
        #print("111 Binary classification?", binary_class)

        # Perform a one-way ANOVA and calculate the p-value
        p_value = 1.0
        if binary_class:
            _, p_value = ttest_ind(X[y==0], X[y==1])  # Assuming binary classification 
            #print("222 Binary classification?", binary_class)
        else: 
            groups = [X[y == label] for label in np.unique(y)] # For multiclass classification 
            _, p_value = f_oneway(*groups)
            #print("222 Not binary classification?", binary_class)
        p_value = p_value[0] 

        # Display the p-value
        #print("P-value:", p_value)

        # Encode the target variable - For multiclass 
        if not binary_class: 
            label_encoder = LabelEncoder()
            y = label_encoder.fit_transform(y)
            #print("333 Not binary classification?", binary_class)

        # Fit a logistic regression model and calculate the AUC
        model = None 
        #if binary_class:
        #    model = LogisticRegression()
        #else:
        #    model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
        # model = LogisticRegression()
        # model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
        model = SVC(probability=True)
        # model = SVC(C=1.0, random_state=1, kernel='linear', probability=True)
        model.fit(X, y)
        y_pred_proba = model.predict_proba(X)
        if binary_class: 
            y_pred_proba = y_pred_proba[:, 1]
            #print("444 Binary classification?", binary_class)

        # print(y_pred_proba) 
        auc = 0.0 
        if binary_class:
            auc = roc_auc_score(y, y_pred_proba)
            #print("555 Binary classification?", binary_class)
        else:
            auc = roc_auc_score(y, y_pred_proba, multi_class='ovr')
            #print("555 Not binary classification?", binary_class)

        # Display the AUC
        #print("AUC:", auc)
        return p_value, auc 
    
    def calculate_p_and_auc_for_dataset(self, all_feats_df, label_df, binary_class=True): 
        feat_cols = all_feats_df.columns.values.tolist() 

        all_p_list = [] 
        all_auc_list = [] 
        for ft in feat_cols:
            feat_data = all_feats_df[[ft]].copy() 
            label_data = label_df.copy() 
            # print("HHHHHH", feat_data.shape, type(feat_data), label_data.shape, type(label_data), binary_class)
            p, auc = self.calculate_p_and_auc_for_feature(feat_data, label_data, binary_class=binary_class) 
            all_p_list.append(p) 
            all_auc_list.append(auc) 

        all_p_and_auc_df = pd.DataFrame( {"Features": feat_cols, f"P_Value_{'bin' if binary_class else 'multi'}": all_p_list, f"AUC_{'bin' if binary_class else 'multi'}": all_auc_list} )    
        return all_p_and_auc_df 
    
    def get_selected_feature_list_based_on_PAUC(self, tmp_df, p_threshold=0.05, auc_threshold=0.5, sort=False, binary_class=True): 
        cols = tmp_df['Features'].values.tolist() 
        if p_threshold:
            tmp_df = tmp_df[(tmp_df[f"P_Value_{'bin' if binary_class else 'multi'}"]<p_threshold)]
        if auc_threshold:
            tmp_df = tmp_df[(tmp_df[f"AUC_{'bin' if binary_class else 'multi'}"]>auc_threshold)]
        if sort:
            tmp_df = tmp_df.sort_values([f"P_Value_{'bin' if binary_class else 'multi'}", f"AUC_{'bin' if binary_class else 'multi'}"], ascending = [True, False])
        selected_features = tmp_df['Features'].values.tolist() 
        return selected_features
    
    def select_pandauc_based_features(self, all_feats_df, label_df, binary_class=True, p_threshold=None, auc_threshold=None, sort=False): 
        tmp_df = self.calculate_p_and_auc_for_dataset(all_feats_df, label_df, binary_class=binary_class)
        selected_features = self.get_selected_feature_list_based_on_PAUC(tmp_df, p_threshold=p_threshold, auc_threshold=auc_threshold, sort=sort, binary_class=binary_class)        
        return selected_features, tmp_df 
    

In [158]:
### Custom splitter class 
import math
from sklearn.model_selection import BaseCrossValidator

# class MyCustomSplitter(BaseCrossValidator):
class MyCustomSplitter():
#     def __init__(self, n_splits):
#         self.n_splits = n_splits
        
#     def set_criteria(self, splt_cri, groups=[['n'], ['SC', 'ST']]):
#         self.splt_cri = splt_cri
#         self.groups = groups
        
    def __init__(self, splt_cri, groups=[['n'], ['SC', 'ST']]):
        self.splt_cri = splt_cri
        self.groups = groups

    def split(self, x, y=None):
        groups = self.groups 
        fold = self.splt_cri[0] 
        percent = self.splt_cri[1]
        y = np.array(y) 
        unique_y = np.unique(y)
        num_y = len(unique_y)
        y_indices = np.arange(num_y)
        # print('AAAA--->> ', unique_y, y_indices)

        remain_x = [i for i,v in enumerate(x)]
        all_filterred_x = []
        for grp in groups:
#             print('Group', grp)
            tmp_filterred_x = [] 
            for it in grp: 
                gg = f"^{it}\d"
                r = re.compile(gg) 
                filterred_x = list(filter(r.match, x))
                # print('BBBB--->> ', gg, filterred_x) 
                filterred_x_ind = [i for i,v in enumerate(x) if v in filterred_x]
                tmp_filterred_x.extend(filterred_x_ind) 
                # tmp_filterred_x.extend(filterred_x) 
            remain_x = [i for i in remain_x if i not in tmp_filterred_x] 
            all_filterred_x.append(tmp_filterred_x) 
            # print('CCCC--->> ', all_filterred_x)
            # print('222--->', grp, remain_x) 

        # print('--->', remain_x) 
        remain_x_ind = [i for i,v in enumerate(x) if i in remain_x]
        all_filterred_x.append(remain_x_ind) 
        # all_filterred_x.append(remain_x)     
        all_dat = [item for row in all_filterred_x for item in row]
        # print('DDDD--->> ', all_filterred_x, all_dat)

        num_groups = len(all_filterred_x) 
        groups_item_len = [len(it) for it in all_filterred_x] 
        groups_item_ratio = [int(it/fold) if (it/fold)==int(it/fold) else int(it/fold+1) for it in groups_item_len] 
        # print('EEEE--->> ', num_groups, groups_item_len, groups_item_ratio)

        main_grps = [it//fold  for it in groups_item_len]  # math.floor(it/fold) 
        ext_grps = [it%fold for it in groups_item_len] 
        # print('FFFF--->> ', main_grps, ext_grps)  

        all_fold_values = [[] for _ in range(fold)] 
        for i, dat in enumerate(all_fold_values):
            test_dat = [] 
            for l, (j,k,fd) in enumerate(zip(main_grps, ext_grps, all_filterred_x)): 
                # print('---->> ', i, j, k, i*j, i*j+j*1, i<k, fold*j+i)
                dd = fd[i*j : i*j+j*1]
                ex_dd = [fd[fold*j+i]] if i<k else []
                dd.extend(ex_dd)
                # print('---->> ', i, j, k, i*j, i*j+j*1, i<k, fold*j+i, dd, ex_dd)
                test_dat.extend( dd ) 
            test_dat = list(set(test_dat))
            train_dat = list( set(all_dat)-set(test_dat) ) 
            # print('GGGG--->> ', test_dat, train_dat)
            yield train_dat, test_dat 

In [162]:
### ML Classifier class 
##### import random
import multiprocessing as mp

from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, ShuffleSplit, LeavePOut, KFold, ParameterGrid

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier 


from sklearn.model_selection import GridSearchCV, RandomizedSearchCV



###########################################################
# HumachLab_ML_CLassifiers     



# ### All models' implementation

class HumachLab_ML_CLassifiers:
    
    def print_message(self):
#         ---------------------------------------------------------------------------------------------------
#         ===================================================================================================
#         ###################################################################################################
#         ***************************************************************************************************
        self.logger.info(f"Hello from HumachLab_ML_CLassifiers class")
    
    
    
    def __init__(self, logger, directory, dataset, class_name, label_map, metadata_column, split_column, random_state_value, split_balance_pattern, check_result=False): 
        self.logger = logger 
        self.directory = directory
        self.dataset = dataset 
        self.class_name = class_name
        self.label_map = label_map
        self.metadata_column = metadata_column
        self.split_column = split_column 
        self.is_multiclass = True if len(dataset[class_name].unique().tolist())>2 else False
        self.random_state_value = random_state_value
        self.split_balance_pattern = split_balance_pattern
        if not check_result:
            self.experiment_info = {
                'logger':logger, 'directory':directory, 'dataset_size':dataset.shape, 'dataset_columns':dataset.columns.values.tolist(), 'metadata_column':metadata_column, 
                'class_name':class_name, 'label_map':label_map, 'split_column':split_column, 'total_unique_classes':dataset[class_name].value_counts().keys().tolist(), 
                'total_unique_classes':dataset[class_name].value_counts().values.tolist()
                }
        
        self.best_model_scoring_metrics=[ML_Performace_Metrics.RECL, ML_Performace_Metrics.F1SCR] 
        
        self.logger.info(f"""
        Object is initialised with the following properties: 
        ###################################################################################################
        Dataset size: {self.dataset.shape}, Columns: {self.dataset.columns.values.tolist()}
        Target class column name: {self.class_name}
        Metadata column names: {self.metadata_column}
        Dataset split column on which the training and test sets will be devided: {self.split_column}
        Is multi-class classification: {self.is_multiclass}
        """) 
        return  
    
    
    def convert_list_to_string(self, lst):
        lst = [str(l) for l in lst]        
        return '* '.join(lst) 
    
    
    
    def classify(self, should_use_params, splitting_crieteria, model_list, is_validate_models, result_save_path, exp_name, exp_detail, apply_feature_selection, custom_splitter):
        self.splitting_crieteria = splitting_crieteria    ### for test & training (validation) splitting_crieteria (n): n=0 -loso, n>0 -n-fold, n<0 -shuffled random splitting with n% testing
        self.model_list = model_list 
        self.should_use_params = should_use_params
        self.is_validate_models = is_validate_models 
        self.result_save_path = result_save_path 
        self.exp_name = exp_name 
        self.exp_detail = exp_detail 
        self.is_binary_classification = not self.is_multiclass
        self.apply_feature_selection = apply_feature_selection 
        self.selected_features = None 
        self.custom_splitter = custom_splitter
        
        # self.experiment_info['exp_name'] = exp_name 
        self.experiment_info.update(exp_detail)
        self.experiment_info['apply_feature_selection'] = apply_feature_selection
        #self.experiment_info['selected_features'] = self.selected_features 
        self.experiment_info['is_multiclass_classification'] = self.is_multiclass
        self.experiment_info['model_list'] = model_list 
        self.experiment_info['should_use_params'] = should_use_params
        self.experiment_info['is_validate_models'] = is_validate_models
        self.experiment_info['result_save_path'] = result_save_path
        self.experiment_info['random_state_value'] = self.random_state_value 
        self.experiment_info['custom_splitter'] = self.custom_splitter
        self.experiment_info['split_balance_pattern'] = self.split_balance_pattern
        tmp = splitting_crieteria[0] 
        self.experiment_info['test_split_crieteria'] = tmp 
        self.experiment_info['test_split_details'] = f'Leave-one-out' if tmp[0]==0 else (f'{tmp[0]}-fold cross validation' if (tmp[0]>0 and tmp[1]<=0) else f'{tmp[0]}-fold {tmp[1]}% random test splitting') 
        tmp = splitting_crieteria[1] 
        self.experiment_info['training_split_crieteria'] = tmp 
        self.experiment_info['training_split_details'] = f'Leave-one-out' if tmp[0]==0 else (f'{tmp[0]}-fold cross validation' if (tmp[0]>0 and tmp[1]<=0) else f'{tmp[0]}-fold {tmp[1]}% random test splitting') 
        self.experiment_info['model_selection_matrics'] = self.best_model_scoring_metrics 
                                                                                                  
                                                                                                                      
        self.logger.info(f"""
        Classification is set with the following parameters: 
        ###################################################################################################
        Splitting crieteria: {self.splitting_crieteria}
        Test split: {f'Leave-one-out' if splitting_crieteria[0] [0]==0 else (f'{splitting_crieteria[0] [0]}-fold cross validation' if (splitting_crieteria[0] [0]>0 and splitting_crieteria[0] [1]<=0) else f'{splitting_crieteria[0] [0]}-fold {splitting_crieteria[0] [1]}% random test splitting') }
        Training split: {f'Leave-one-out' if splitting_crieteria[1] [0]==0 else (f'{splitting_crieteria[1] [0]}-fold cross validation' if (splitting_crieteria[1] [0]>0 and splitting_crieteria[1] [1]<=0) else f'{splitting_crieteria[1] [0]}-fold {splitting_crieteria[1] [1]}% random test splitting') }
        List of ML models that will be applied: {[mn.value for mn in self.model_list]}
        Use parameters for model: {self.should_use_params}
        Is validate the model (or only train): {self.is_validate_models} 
        Classification results will be saved in the directory: {self.result_save_path}
        """) 
        all_exp_info_df = pd.DataFrame(self.experiment_info.items(), columns=['Information', 'Description']) 
        all_best_tr_model, all_tr_model, all_tr_scores_df, all_tr_prediction_df, all_ts_model, all_ts_scores_df, all_ts_prediction_df, all_ts_fold_info_df  = self.test() 
        
        self.save_results(self.directory, all_best_tr_model, all_tr_model, all_tr_scores_df, all_tr_prediction_df, all_ts_model, all_ts_scores_df, all_ts_prediction_df, all_ts_fold_info_df, all_exp_info_df) 
        
        return all_best_tr_model, all_tr_model, all_tr_scores_df, all_tr_prediction_df, all_ts_model, all_ts_scores_df, all_ts_prediction_df, all_ts_fold_info_df, all_exp_info_df 
    
    
    
    def load_results(self, save_directory):
        # './Results/_Classification/ML001/'
        self.logger.info(f"""
        Data is being loaded from: {save_directory}
        """) 
        save_path = f"{save_directory}all_tr_scores.csv" 
        all_tr_scores_df = pd.read_csv(save_path, index_col=False) 
        
        save_path = f"{save_directory}all_tr_prediction.csv" 
        all_tr_prediction_df = pd.read_csv(save_path, index_col=False) 
        
        save_path = f"{save_directory}all_ts_scores.csv" 
        all_ts_scores_df = pd.read_csv(save_path, index_col=False) 
        
        save_path = f"{save_directory}all_ts_prediction.csv" 
        all_ts_prediction_df = pd.read_csv(save_path, index_col=False)         
        
        save_path = f"{save_directory}all_ts_fold_info.csv" 
        all_ts_fold_info_df = pd.read_csv(save_path, index_col=False)         
        
        save_path = f"{save_directory}all_exp_info.csv" 
        all_exp_info_df = pd.read_csv(save_path, index_col=False)  
        
        new_save_directory = f"{save_directory}/Models/"
        
        save_path = f"{new_save_directory}ts_model" 
        all_ts_model = self.load_models_from_file(save_path, 'Test Models')
        
        save_path = f"{new_save_directory}tr_model" 
        all_tr_model = self.load_models_from_file(save_path, 'Training Models')
        
        save_path = f"{new_save_directory}best_tr_model" 
        all_best_tr_model = self.load_models_from_file(save_path, 'Best Training Models')
        
        return all_best_tr_model, all_tr_model, all_tr_scores_df, all_tr_prediction_df, all_ts_model, all_ts_scores_df, all_ts_prediction_df, all_ts_fold_info_df, all_exp_info_df 


    def load_models_from_file(self, save_path, model_type):
        models_dict = {} 
        
        save_path = f'{save_path}*'
        files = self.sort_string_list(glob.glob(save_path)) 
        files
        selected_files = [[int(fn) for fn in f[len(save_path):].split('.')[0].split('_')] for f in files]
        selected_files
        
        self.logger.info(f'Start retrieving {model_type} model from file...')
        model_dict = {}  
        for i, (ind, fl) in enumerate(zip(selected_files, files)):
            if len(ind)==3:
                tsi, tri, modi = ind
                mod = None 
                with open(fl, "rb") as f:
                    mod = pickle.load(f) 
                if tsi not in model_dict.keys(): 
                    model_dict[tsi] = {}
                if tri not in model_dict[tsi].keys(): 
                    model_dict[tsi][tri] = {} 
                if modi not in model_dict[tsi][tri].keys(): 
                    model_dict[tsi][tri][modi] = mod 
            elif len(ind)==2:
                tsi, modi = ind
                mod = None 
                with open(fl, "rb") as f:
                    mod = pickle.load(f) 
                if tsi not in model_dict.keys(): 
                    model_dict[tsi] = {}
                if modi not in model_dict[tsi].keys(): 
                    model_dict[tsi][modi] = mod 
            else:
                self.logger.info(f'Doesn\'t identify {model_type} model file to retrieve...')
        
        model_dict
        self.logger.info(f'Finish retrieving {model_type} model from file...')
        return model_dict 
    
    
    
    def sort_string_list(self, string_list):
        ## ref: https://blog.codinghorror.com/sorting-for-humans-natural-sort-order/
        """ Sort the given list in the way that humans expect.
        """
        convert = lambda text: int(text) if text.isdigit() else text
        alphanum_key = lambda key: [ convert(c.replace("_","")) for c in re.split('([0-9]+)', key) ]
        string_list.sort( key=alphanum_key )
        return string_list
    
    
        
    def save_results(self, save_directory, all_best_tr_model, all_tr_model, all_tr_scores_df, all_tr_prediction_df, all_ts_model, all_ts_scores_df, all_ts_prediction_df, all_ts_fold_info_df, all_exp_info_df):
        # './Results/_Classification/ML001/'
        save_path = f"{save_directory}all_tr_scores.csv" 
        all_tr_scores_df.to_csv(save_path, index=False) 
        
        save_path = f"{save_directory}all_tr_prediction.csv" 
        all_tr_prediction_df.to_csv(save_path, index=False) 
        
        save_path = f"{save_directory}all_ts_scores.csv" 
        all_ts_scores_df.to_csv(save_path, index=False) 
        
        save_path = f"{save_directory}all_ts_prediction.csv" 
        all_ts_prediction_df.to_csv(save_path, index=False) 
        
        save_path = f"{save_directory}all_ts_fold_info.csv" 
        all_ts_fold_info_df.to_csv(save_path, index=False) 
        
        save_path = f"{save_directory}all_exp_info.csv" 
        all_exp_info_df.to_csv(save_path, index=False) 
        
        new_save_directory = self.create_directory(save_directory, 'Models')
        
        save_path = f"{new_save_directory}ts_model" 
        self.save_models_to_file(save_path, all_ts_model, 'Test Models')
        
        save_path = f"{new_save_directory}tr_model" 
        self.save_models_to_file(save_path, all_tr_model, 'Training Models')
        
        save_path = f"{new_save_directory}best_tr_model" 
        self.save_models_to_file(save_path, all_best_tr_model, 'Best Training Models')
        
        return
    
    
    def create_directory(self, path, dir_name): 
        new_directory = f"{path}/{dir_name}/"
        
        if (not os.path.exists(new_directory)):
            try:
                os.makedirs(new_directory, exist_ok = True)
                print(f"Directory successfully created at path: {new_directory}") 
            except OSError as error:
                print(f"Directory cannot be created at path: {new_directory}") 
        else:
            print(f"Directory already exists at path: {new_directory}") 
            
        return new_directory


    def save_models_to_file(self, save_path, models_dict, model_type):
        self.logger.info(f'Start saving model to file...')
        if model_type=='Training Models':
            for tsfi, ts_dat in models_dict.items():
                for trfi, tr_dat in ts_dat.items():
                    for modi, mod in tr_dat.items():
                        new_save_path = f'{save_path}_{tsfi}_{trfi}_{modi}.dat'
                        try:
                            with open(new_save_path, 'wb') as f:
                                pickle.dump(mod, f)
                                print( f'{model_type} is written to the file: {new_save_path}\n' )
                        except:
                            print( f'Problem creating {model_type} file: {new_save_path}\n' )
        else:
            for tsfi, ts_dat in models_dict.items():
                for modi, mod in ts_dat.items():
                    new_save_path = f'{save_path}_{tsfi}_{modi}.dat'
                    try:
                        with open(new_save_path, 'wb') as f:
                            pickle.dump(mod, f)
                            print( f'{model_type} is written to the file: {new_save_path}\n' )
                    except:
                        print( f'Problem creating {model_type} file: {new_save_path}\n' )
        self.logger.info(f'Finish saving model to file...')
        return
    
    
    
    def evaluate_test_data(self, ind, model_list, test_ids, best_tr_model):
        self.logger.info(f"""
        ### MODEL EVALUATION PHASE 
        EVALUATION {ind} START... XXXXX 
        ---------------------------------------------------------------------------------------------------
        """) 
        #print("HHHHHH3333", best_tr_model)
        
        X_test, y_test, meta_dat_df, _ = self._get_data_from_indices(test_ids, from_where=f'_TS_ts_{ind}')
        #print("meta33->", meta_dat_df)     
#         ### Call features selection algorithm 
#         if self.selected_features: 
#             X_test = X_test[self.selected_features]
        
        
        ts_score_df, ts_prediction_df = pd.DataFrame(), pd.DataFrame() 
        # all_ts_model, all_ts_scores_df, all_ts_prediction_df = {}, pd.DataFrame(), pd.DataFrame()
        for (modi, model), classifier_method in zip(best_tr_model.items(), model_list) : 
            y_pred = model.predict(X_test) 
            y_pred_proba = model.predict_proba(X_test) 
            if modi==1:
                # meta_dat_df.reset_index(drop=True, inplace=True) 
                ts_prediction_df = pd.concat([ts_prediction_df, meta_dat_df]) 
                ts_prediction_df.reset_index(drop=True, inplace=True) 
                ts_prediction_df[self.class_name] = y_test 
                
            ts_prediction_df[f"Prediction_{ML_Classifiers.get_short_form(str(classifier_method.value))}"] = y_pred 
            for p_ind in range(y_pred_proba.shape[1]):
                ts_prediction_df[f"Prediction_Proba_{p_ind}_{ML_Classifiers.get_short_form(str(classifier_method.value))}"] = y_pred_proba[:, p_ind].tolist()
            
            scores_df = self.calculate_model_scores(model, y_test, y_pred, y_pred_proba)
            scores_df.insert(0, "Model_No", modi) 
            ts_score_df = pd.concat([ts_score_df, scores_df]) 
            
        ts_score_df.insert(0, "Test_No", ind) 
        ts_prediction_df.insert(0, "Test_No", ind) 
        return ts_score_df, ts_prediction_df 
    
    
    
    def test(self):
        # splitter = self.get_data_splitter(self.splitting_crieteria[0], stratified=True, custom=True) #Test split cusomised: usually LOSO or 10-fold 
        # splitter = self.get_data_splitter(self.splitting_crieteria[0], stratified=True, custom=False) #Test split: usually LOSO or 10-fold 
        splitter = self.get_data_splitter(self.splitting_crieteria[0], stratified=True, custom=self.custom_splitter) #Test split cusomised: usually LOSO or 10-fold 
        split_data_list = self.dataset[self.split_column].values.tolist() 
        class_data_list = self.dataset[self.class_name].values.tolist() 
        
        all_ts_model, all_ts_scores_df, all_ts_prediction_df, all_ts_fold_info_df = {}, pd.DataFrame(), pd.DataFrame(), pd.DataFrame()  
        cum_best_tr_model, cum_tr_model, cum_tr_scores_df, cum_tr_prediction_df, cum_tr_fold_info_df = {}, {}, pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
        for tsi, (train_all_ids, test_ids) in enumerate(splitter.split(split_data_list, class_data_list)): 
            ### Train-test split: fold based 
#             ts_dat = self.dataset[self.split_column][test_ids].values.tolist() 
#             tr_all_dat = self.dataset[self.split_column][train_all_ids].values.tolist() 
#             ts_dat = self.dataset.loc[test_ids, self.split_column].values.tolist() 
#             tr_all_dat = self.dataset.loc[train_all_ids, self.split_column].values.tolist() 
            ts_dat = self.dataset.iloc[test_ids][self.split_column].values.tolist() 
            tr_all_dat = self.dataset.iloc[train_all_ids][self.split_column].values.tolist() 
            print('test- tsi, train_all_ids, test_ids, tr_all_dat, ts_dat', tsi, train_all_ids, test_ids, tr_all_dat, ts_dat) 
#             print(f"QQQQQQQQQQQQQ {train_all_ids}, {self.dataset[self.split_column][train_all_ids].values.tolist()}, {self.dataset[self.split_column].values.tolist() }")
            ind = tsi+1 
            self.selected_features = None ### Resetting feature selection list 
            self.logger.info(f"""
            ### MODEL TEST PHASE 
            TEST {ind} START... XXXXX 
            ===================================================================================================
            Test=> {len(ts_dat)} {(ts_dat)} 
            Training (Including Validation)=> {len(tr_all_dat)} {(tr_all_dat)} 
            """) 
            best_tr_model, all_tr_model, all_tr_scores_df, all_tr_prediction_df, all_tr_fold_info_df = self.train(ind, model_list, train_all_ids, tr_splitting_crieteria=self.splitting_crieteria[1]) #tr_splitting_crieteria: <0 random split, >0 fold
#             continue
            cum_tr_model[ind] = all_tr_model 
            cum_best_tr_model[ind] = best_tr_model 
            if self.is_validate_models:
                all_tr_scores_df.insert(0, "Test_No", ind) 
                all_tr_prediction_df.insert(0, "Test_No", ind) 
            all_tr_fold_info_df.insert(0, "Test_No", ind) 
            all_tr_fold_info_df.insert(4, "Test", [ts_dat]*all_tr_fold_info_df.shape[0]) 
            all_tr_fold_info_df.insert(4, "Selected_Features", [self.selected_features]*all_tr_fold_info_df.shape[0]) 
            
            cum_tr_scores_df = pd.concat([cum_tr_scores_df, all_tr_scores_df]) 
            cum_tr_prediction_df = pd.concat([cum_tr_prediction_df, all_tr_prediction_df])
            all_ts_fold_info_df = pd.concat([all_ts_fold_info_df, all_tr_fold_info_df])
            
            cum_tr_scores_df.reset_index(drop=True, inplace=True) 
            cum_tr_prediction_df.reset_index(drop=True, inplace=True) 
            all_ts_fold_info_df.reset_index(drop=True, inplace=True) 
            
#             print("TTTT", best_tr_model.keys(), best_tr_model, all_tr_scores_df.shape, all_tr_scores_df.columns, all_tr_prediction_df.shape, all_tr_prediction_df.columns) 
                        
            ###############
            ### Model evaluation with the test data using the best trained model  
            all_ts_model[ind] = best_tr_model
            ts_score_df, ts_prediction_df = self.evaluate_test_data(ind, model_list, test_ids, best_tr_model) 
            
            all_ts_scores_df = pd.concat([all_ts_scores_df, ts_score_df]) 
            all_ts_prediction_df = pd.concat([all_ts_prediction_df, ts_prediction_df])
            
            all_ts_scores_df.reset_index(drop=True, inplace=True) 
            all_ts_prediction_df.reset_index(drop=True, inplace=True) 
        
            self.logger.info(f"""
            ===================================================================================================
            TEST {ind} END...
            """) 
            
        ### Sorting scores
#             print( 'TTRR', cum_tr_scores_df.columns.values.tolist(), cum_tr_prediction_df.columns.values.tolist() )
        cum_tr_scores_df.sort_values(['Model_No', 'Test_No', 'Training_No'], ascending = [True, True, True], inplace=True)  
        cum_tr_prediction_df.sort_values(['Test_No', 'Training_No'], ascending = [True, True], inplace=True)  

#             print( 'TTSS', all_ts_scores_df.columns.values.tolist(), all_ts_prediction_df.columns.values.tolist() )
        all_ts_scores_df.sort_values(['Model_No', 'Test_No'], ascending = [True, True], inplace=True) 
        all_ts_prediction_df.sort_values(['Test_No'], ascending = [True], inplace=True) 
        all_ts_fold_info_df.sort_values(['Model_No', 'Test_No', 'Training_No'], ascending = [True, True, True], inplace=True) 
        
        all_ts_fold_info_df
        
        return cum_best_tr_model, cum_tr_model, cum_tr_scores_df, cum_tr_prediction_df, all_ts_model, all_ts_scores_df, all_ts_prediction_df, all_ts_fold_info_df  
        
    
    
    
    def train(self, tsi, model_list, train_all_ids, tr_splitting_crieteria):           
        # tr_splitter = self.get_data_splitter(tr_splitting_crieteria, stratified=True, custom=True) #Training split customised: usually 5-fold or Random 20% split
        tr_splitter = self.get_data_splitter(tr_splitting_crieteria, stratified=True, custom=False) #Training split: usually 5-fold or Random 20% split
        all_tr_model, all_tr_scores_df, all_tr_prediction_df, all_tr_fold_info_df = {}, pd.DataFrame(), pd.DataFrame(), pd.DataFrame()  
        best_tr_model, best_acc, best_rec, best_prec = {}, [], [], []  
        class_data_list2 = self.dataset.iloc[train_all_ids][self.class_name].values.tolist() 
        split_data_list2 = self.dataset.iloc[train_all_ids][self.split_column].values.tolist() 
#         split_data_list2 = self.dataset[self.split_column][train_all_ids].values.tolist()
#         class_data_list2 = self.dataset[self.class_name][train_all_ids].values.tolist() 
#         class_data_list2 = self.dataset.loc[train_all_ids, self.class_name].values.tolist() 
#         split_data_list2 = self.dataset.loc[train_all_ids, self.split_column].values.tolist()
#         print(f"QQQQQQQQQQQQQ 222222 {train_all_ids}, {self.dataset[self.split_column][train_all_ids].values.tolist()}, {self.dataset[self.split_column].values.tolist()}")
        # print('QQQQQQQQQQQ', split_data_list2, class_data_list) 
        for tri, (tmp_train_ids, tmp_val_ids) in enumerate(tr_splitter.split(split_data_list2, class_data_list2)): 
        # for tri, (tmp_train_ids, tmp_val_ids) in enumerate(tr_splitter.split(split_data_list2)): 
            
#             if not should_validate: 
#                 tmp_train_ids = np.concatenate((tmp_train_ids, tmp_val_ids)) 
#             else:
#                 tmp_train_ids = np.concatenate((tmp_train_ids, tmp_val_ids)) 
#                 tmp_val_ids = tmp_train_ids.copy() 
            tmp_train_ids = np.concatenate((tmp_train_ids, tmp_val_ids)) 
            tmp_val_ids = tmp_train_ids.copy() 
            
            ### This tmp_train_ids and tmp_val_ids are indexes re-starting from 0 again, but we need to get the actual indexes in the dataset 
            xx = (np.array(split_data_list2)[tmp_val_ids]).tolist() 
            val_ids = self.dataset[self.dataset[self.split_column].isin(xx)].index.tolist()
            xx = (np.array(split_data_list2)[tmp_train_ids]).tolist() 
            train_ids = self.dataset[self.dataset[self.split_column].isin(xx)].index.tolist()
            
            ### Validation-train split: random percentage based 
            val_dat = self.dataset[self.split_column][val_ids].values.tolist() 
            tr_dat = self.dataset[self.split_column][train_ids].values.tolist() 
#             val_dat = self.dataset.loc[val_ids, self.split_column].values.tolist() 
#             tr_dat = self.dataset.loc[train_ids, self.split_column].values.tolist() 
#             val_dat = self.dataset.iloc[val_ids][self.split_column].values.tolist() 
#             tr_dat = self.dataset.iloc[train_ids][self.split_column].values.tolist() 
            print('train- tri, tmp_train_ids, tmp_val_ids, tr_dat, val_dat', tri, tmp_train_ids, tmp_val_ids, tr_dat, val_dat) 
            ind = tri+1 
            self.logger.info(f"""
            *** MODEL TRAINING AND VALIDATION PHASE FOR TEST {tsi} 
            TRAINING {tri+1} START... XXXXX 
            ***************************************************************************************************
            Validation=> {len(val_dat)} {(val_dat)} 
            Training=> {len(tr_dat)} {(tr_dat)} 
            """)            
#             continue
            all_mtr_model, all_mtr_scores_df, all_mtr_prediction_df, all_mtr_fold_info_df = self.train_models(model_list, train_ids, val_ids, tsi, ind)            
            all_tr_model[ind] = all_mtr_model 
            if self.is_validate_models:
                all_mtr_scores_df.insert(0, "Training_No", ind) 
                all_mtr_prediction_df.insert(0, "Training_No", ind) 
            all_mtr_fold_info_df.insert(0, "Training_No", ind) 
                
            all_tr_scores_df = pd.concat([all_tr_scores_df, all_mtr_scores_df]) 
            all_tr_prediction_df = pd.concat([all_tr_prediction_df, all_mtr_prediction_df])
            all_tr_fold_info_df = pd.concat([all_tr_fold_info_df, all_mtr_fold_info_df])
            
            all_tr_scores_df.reset_index(drop=True, inplace=True) 
            all_tr_scores_df.reset_index(drop=True, inplace=True)    
            all_tr_fold_info_df.reset_index(drop=True, inplace=True)          
            
            self.logger.info(f"""
            ---------------------------------------------------------------------------------------------------
            Best model index calculation  
            """)             
#             print("PPPPP", tri, ind, all_mtr_model.keys(), all_mtr_model, all_mtr_scores_df)
            if tri==0:
                best_tr_model = all_mtr_model.copy() 
                print("WWWWWWWWWWWWWWWWWWW", ML_Performace_Metrics.RECL.value, all_mtr_scores_df)
                #print("HHHH", all_mtr_scores_df.columns)
                # best_acc, best_prec, best_rec = all_mtr_scores_df[ML_Performace_Metrics.ACC.value], all_mtr_scores_df[ML_Performace_Metrics.PREC.value], all_mtr_scores_df[ML_Performace_Metrics.RECL.value] 
                best_rec = all_mtr_scores_df[ML_Performace_Metrics.RECL.value].values.tolist()  
            else:                
                for jj, mn in enumerate(model_list):
                    mod_name = ML_Classifiers.get_short_form(str(mn.value))
                    tm_df = all_mtr_scores_df[(all_mtr_scores_df["Model_Name"]==mod_name)] 
                    new = tm_df[ML_Performace_Metrics.RECL.value].values.tolist()[0] 
                    if new>best_rec[jj]: 
                        best_rec[jj] = new 
                        best_tr_model[tm_df["Model_No"].values.tolist()[0]] = all_mtr_model[tm_df["Model_No"].values.tolist()[0]]  
                        
            
            self.logger.info(f"""
            ***************************************************************************************************
            TRAINING {ind} END... 
            """) 
        return best_tr_model, all_tr_model, all_tr_scores_df, all_tr_prediction_df, all_tr_fold_info_df 
    
    
    
    def train_models(self, model_list, train_ids, val_ids, ts_serial, tr_serial):
        all_mtr_model, all_mtr_scores_df, all_mtr_prediction_df, all_mtr_fold_info_df = {}, pd.DataFrame(), pd.DataFrame(), pd.DataFrame() 
        for modi, classifier_method in enumerate(model_list): 
            ind = modi+1 
            self.logger.info(f"""
            *** ML MODEL FOR TEST:{ts_serial}, TRAINING:{tr_serial} AND MODEL: {ML_Classifiers.get_short_form(str(classifier_method.value))} 
            ---------------------------------------------------------------------------------------------------
            """)
            mtr_model, mtr_scores_df, mtr_prediction_df, mtr_fold_info_df = self.start_training(classifier_method, train_ids, val_ids, ts_serial=ts_serial, tr_serial=tr_serial, mod_serial=ind) 
            #print("HELLO2222", mtr_model, mtr_scores_df, mtr_prediction_df) 
            all_mtr_model[ind] = mtr_model 
            #all_mtr_model["Model_Name"] = ML_Classifiers.get_short_form(str(classifier_method.value))  
            if self.is_validate_models:                   
                if modi>0:
                    mtr_prediction_df.drop(self.metadata_column, axis=1, inplace=True)
                    mtr_prediction_df.drop([self.class_name], axis=1, inplace=True)                
                               
                mtr_scores_df.insert(0, "Model_No", ind) 
                mtr_scores_df.insert(1, "Model_Name", ML_Classifiers.get_short_form(str(classifier_method.value)))  
#                 mtr_scores_df.insert(2, "Selected_Features", [self.selected_features]*mtr_scores_df.shape[0]) 
                    
                all_mtr_scores_df = pd.concat([all_mtr_scores_df, mtr_scores_df]) 
                all_mtr_prediction_df = pd.concat([all_mtr_prediction_df, mtr_prediction_df], axis=1) 
                
                all_mtr_scores_df.reset_index(drop=True, inplace=True) 
                all_mtr_prediction_df.reset_index(drop=True, inplace=True)

            mtr_fold_info_df.insert(0, "Model_No", ind)    
            mtr_fold_info_df.insert(1, "Model_Name", ML_Classifiers.get_short_form(str(classifier_method.value))) 
            all_mtr_fold_info_df = pd.concat([all_mtr_fold_info_df, mtr_fold_info_df]) 
            all_mtr_fold_info_df.reset_index(drop=True, inplace=True)  
                    
        #print("HELLO", all_mtr_model, all_mtr_scores_df, all_mtr_prediction_df) 
        return all_mtr_model, all_mtr_scores_df, all_mtr_prediction_df, all_mtr_fold_info_df
    
    
    
    def get_data_splitter(self, split_crit_tuple, stratified=False, custom=False):
        
        if custom:
            self.logger.info(f"Custom splitter testing...") 
#             splitter = MyCustomSplitter(n_splits=5)
#             splitter.set_criteria(split_crit_tuple) 
            splitter = MyCustomSplitter(split_crit_tuple, groups=self.split_balance_pattern) ### , groups=[['n'], ['SC', 'ST']] for binary and [['n'], ['SC', 'ST']] for multi-class 
            return splitter
        
        spl_rand = self.random_state_value ##random.randint(1, 1000)
        splitter = None
        split_crit = split_crit_tuple[0] ### Fold 
        split_perc = split_crit_tuple[1] ### Fold 
        
        if split_crit==0:
            self.logger.info(f"Leave-one-subject-out testing...") 
            split_num = 1 
            splitter = LeavePOut(p=split_num) if stratified else LeavePOut(p=split_num) 
            # splitter = StratifiedLeavePOut(p=split_num) if stratified else LeavePOut(p=split_num) 
            # splitter = LeavePOut(p=split_num) 
            # splitter = StratifiedLeavePOut(p=split_num) #Stratified
        elif split_crit>0:
            if split_perc<=0:
                self.logger.info(f"{split_crit}-fold testing") 
                split_num = 5
                if split_crit != split_num:
                    split_num = split_crit 
                splitter = StratifiedKFold(n_splits=split_num, shuffle=False) if stratified else KFold(n_splits=split_num, shuffle=False) 
                # splitter = StratifiedKFold(n_splits=split_num, shuffle=True, random_state=spl_rand) if stratified else KFold(n_splits=split_num, shuffle=True, random_state=spl_rand) 
                # splitter = KFold(n_splits=split_num, shuffle=True, random_state=spl_rand)
                # splitter = StratifiedKFold(n_splits=split_num, shuffle=True, random_state=spl_rand)
                #splitter = KFold(n_splits=split_num, random_state=spl_rand)
                #splitter = KFold(n_splits=split_num)
            else:
                split_num = split_crit 
                split_ratio = split_perc 
                self.logger.info(f"Random {split_ratio} percentage splitting testing...") 
                splitter = StratifiedShuffleSplit(n_splits=split_num, test_size=split_ratio, random_state=spl_rand) if stratified else ShuffleSplit(n_splits=split_num, test_size=split_ratio, random_state=spl_rand) 
                # splitter = ShuffleSplit(n_splits=split_num, test_size=split_ratio, random_state=spl_rand) #rs
                # splitter = StratifiedShuffleSplit(n_splits=split_num, test_size=split_ratio, random_state=spl_rand) #rs
                # splitter = ShuffleSplit(n_splits=split_num, test_size=split_ratio) #rs
        else:
            self.logger.info(f"Problem with the splitting with the splitting criteria {split_crit_tuple}...") 
            
        # self.data_splitter = splitter 
        return splitter 
    
    

    def start_training(self, classifier_method, train_ids, val_ids, ts_serial, tr_serial, mod_serial):
        parameters = self.get_parameters_for_ml_models(classifier_method) 
        print("Parameters: ", parameters)
        model, model_scores, target_and_prediction = None, None, None
        model, model_scores, target_and_prediction, fold_info_df = self.call_all_model_optimization(classifier_method, parameters, train_ids, val_ids, ts_serial, tr_serial, mod_serial, parameter_optimization=1)
        return model, model_scores, target_and_prediction, fold_info_df 
    
    
    
    def _get_feature_importance_from_AUC_score(self, indices):       
        self.logger.info(f"""
        Here to calculate feature importance using AUC
        """)
        data = copy.deepcopy(self.dataset).iloc[indices] 
        
        target = data[self.class_name] 
        metadata_df = data[self.metadata_column] 
        features = data.drop([self.class_name]+self.metadata_column, axis=1) 
        
        selected_feats_list, importance_scores = self.select_appropriate_features(features, target, num_features=None, selection_criteria={'auc':0.5}) ### selection_criteria=None/{'auc':0.7} #'corr', 'p', 'auc', 'pandauc' 
        return importance_scores
    
    
    def balance_data(self, features_, target_):
#         oversample = SMOTE()
#         X, y = oversample.fit_resample(X, y)
        tdf = pd.concat([features_, target_], axis=1)
        print()
        dis_dat = tdf[tdf[self.class_name]!=0]
        heal_dat = tdf[tdf[self.class_name]==0]
        dis_cnt = dis_dat.shape[0]
        heal_cnt = heal_dat.shape[0]
        new_df = pd.DataFrame()

        self.logger.info(f'Imbalanced data size: disorders= {dis_dat.shape}, healthy= {heal_dat.shape}')
        
        upsampled_dat = resample(heal_dat, replace=True, n_samples=dis_cnt)
        new_df = pd.concat([dis_dat, upsampled_dat]) 
        
        self.logger.info(f'After balancing data size: disorders= {dis_dat.shape}, healthy= {upsampled_dat.shape}')
        target = tdf[self.class_name] 
        features = tdf.loc[:, tdf.columns != self.class_name]
            
        return features, target


    
    def _get_data_from_indices(self, indices, from_training=False, from_where='_TR_x'):
        data = copy.deepcopy(self.dataset).iloc[indices] 
#         data.to_csv(f"{self.directory}/{from_where}.csv", index=False)
        ### Downsample he raining data: 1=down, 2=up, 3=bound sampling
        #if from_training:
        #    self.logger.info(f'Resampling training data...')
        #    data = self.preprocessor.get_resamplled_data(data, self.class_name, self.pat_id_col, random_sampling=True, up_or_down_sampling=1, min_scale=2.0, max_scale=3.0) ## 0-no, 1-down, 2-up, 3-bound
                    
        self.logger.info(f"""
        From training? {from_training}, Data shape: {data.shape}, Indices: {indices}
        All Columns: {data.columns.values.tolist()}
        """) 
        
        all_cols = data.columns.values.tolist() 
        ft_start_ind = all_cols.index(self.class_name)+1
        
        target = data[self.class_name] 
        metadata_df = data[self.metadata_column] 
#         features = data.drop([self.class_name]+self.metadata_column, axis=1) 
        features = data.iloc[:, ft_start_ind:]
        importance_scores = None
        
        ### Call features selection algorithm 
        if self.apply_feature_selection and from_training and self.selected_features is None: 
            self.logger.info(f""""
            Here comes to feature selection...
            """) 
            selected_feats_list, importance_scores = self.select_appropriate_features(features, target, num_features=None, selection_criteria={'auc':0.5}) ### selection_criteria=None/{'auc':0.7} #'corr', 'p', 'auc', 'pandauc' 
            self.selected_features = selected_feats_list.copy() 
            self.logger.info(f""""
            Selected features: {self.selected_features}
            """) 
        elif from_training and self.selected_features is None: 
            self.selected_features = features.columns.values.tolist() 
            self.logger.info(f""""
            Selected features: {self.selected_features}
            """) 
            
        if 'index' in self.selected_features: #feature_names
            self.selected_features.remove('index')
            
        if self.selected_features is not None:
            # features = features.loc[:, ~features.columns.isin(self.selected_features) ]
            features = features.loc[:, features.columns.isin(self.selected_features) ]
            
            
#         ### Call data balancing using SMOTE 
#         features, target = self.balance_data(features, target) 
        
        
        self.logger.info(f"""
        Feature shape: {features.shape}, Target shape: {target.shape}, Metadata: {metadata_df.shape} 
        """) 
        
        target = target.values.tolist() 
        features = features.values 
        
        return features, target, metadata_df, importance_scores  
    
    
    
    def select_appropriate_features(self, X_dat, y_dat, num_features=None, selection_criteria=None): ### selection_criteria={'auc':0.7} #'corr', 'p', 'auc', 'pandauc' 
        selected_features = [] 
        importance_scores = None 
        crit_name = selection_criteria.keys() 
        crit_name = list(crit_name)[0] 
        dpp_obj = DataPreprocessor() 
        
        if crit_name=='corr':
            pass
        elif crit_name=='p':
            selected_features, importance_scores = dpp_obj.select_pandauc_based_features(X_dat, y_dat, binary_class=self.is_binary_classification, p_threshold=0.05, auc_threshold=None, sort=True) 
        elif crit_name=='auc':
            selected_features, importance_scores = dpp_obj.select_pandauc_based_features(X_dat, y_dat, binary_class=self.is_binary_classification, p_threshold=None, auc_threshold=0.5, sort=True) 
            pass
        elif crit_name=='pandauc':
            selected_features, importance_scores = dpp_obj.select_pandauc_based_features(X_dat, y_dat, binary_class=self.is_binary_classification, p_threshold=0.05, auc_threshold=0.5, sort=True) 
            
        if num_features:
            selected_features = selected_features[:num_features]
        
        return selected_features, importance_scores 
    
    

    def run_model_gridSearch(self, classifier_method, params, train_ids, val_ids, ts_serial, tr_serial, mod_serial):
        tmp_train_ids, tmp_val_ids = train_ids.copy(), val_ids.copy()  
        should_validate = self.is_validate_models 
        # should_validate = True 
        
#         if not should_validate:
#             # tmp_train_ids.extend(tmp_val_ids) 
#             tmp_train_ids = np.concatenate((tmp_train_ids, tmp_val_ids)) 
#         tmp_train_ids = np.concatenate((tmp_train_ids, tmp_val_ids)) 
        
        print('run_model_gridSearch- val_ids, train_ids', val_ids, train_ids)
        ### Validation-train split: random percentage based 
        val_dat = self.dataset[self.split_column][val_ids].values.tolist() 
        tr_dat = self.dataset[self.split_column][train_ids].values.tolist() 
#         val_dat = self.dataset.iloc[val_ids, self.split_column].values.tolist() 
#         tr_dat = self.dataset.iloc[train_ids, self.split_column].values.tolist() 
        print('run_model_gridSearch- val_dat, tr_dat', val_dat, tr_dat) 
        fold_info_df = pd.DataFrame([[val_dat, tr_dat]], columns=['Validation', 'Training']) 
        
        X_train, y_train, _, importance_scores = self._get_data_from_indices(tmp_train_ids, from_training=True, from_where=f'_TR_ts_{ts_serial}_tr_{tr_serial}_mod_{mod_serial}') 
#         if importance_scores is None:
#             importance_scores = self._get_feature_importance_from_AUC_score(tmp_train_ids)
        importance_scores = self._get_feature_importance_from_AUC_score(tmp_train_ids)
        
        mods = self.get_ml_model_instances(classifier_method)
        self.logger.info(f"""
        GridSearch: {ML_Classifiers.get_short_form(str(classifier_method.value))} - {params} 
        ---------------------------------------------------------------------------------------------------
        """)
        parameters = {}
        model = mods
        model_scores = None
        if self.should_use_params:
            parameters = params

        scoring, refit = self.get_ml_scoring_metrices(self.best_model_scoring_metrics[0]) 
#         scoring, refit = 'f1', True
        self.logger.info(f"""Refitting the model with best parameter {scoring} == {refit}""")
        
        model = GridSearchCV(mods, parameters, scoring=scoring, cv=5, refit=refit, return_train_score=True, n_jobs=50, verbose=2)
        # model = GridSearchCV(mods, parameters, scoring=scoring, cv=5, refit=refit, return_train_score=True, error_score='raise', n_jobs=50, verbose=2)
        # model = GridSearchCV(mods, parameters, scoring=scoring, cv=5, refit=refit, return_train_score=True, n_jobs=mp.cpu_count(), verbose=2)

        # ### Scoring from custom method
        # score = make_scorer(self.custom_precision_func, greater_is_better=False)
        # # scoring = {'precision': score, 'f1':make_scorer(f1_score)}
        # model = GridSearchCV(mods, parameters, scoring=score, cv=self.cross_validation_rounds, refit=refit, return_train_score=True, n_jobs=-1, verbose=2)
        
        # X_train = np.nan_to_num(X_train)
        model = model.fit(X_train, y_train) 
        mod = copy.deepcopy(model) 
        mod_est = model.best_estimator_ 
        mod_par = model.best_params_
        
        # print('KKKKKKKKK-->>>', model, mod_est, mod_par)
        model_scores = None 
        target_and_prediction_df = pd.DataFrame() 
        
#         ### Rebuild the model with best parameter         
#         # if should_validate:
#         bst_parameters = model.best_params_
#         self.logger.info(f"""Refitting the model with best parameter""")
#         # mod = mod.set_params(**bst_parameters)
#         mod = mod.best_estimator_.set_params(**bst_parameters)
#         tmp_train_ids2 = np.concatenate((train_ids.copy(), val_ids.copy()))
#         # X_train2, y_train2, _, _ = self._get_data_from_indices(tmp_train_ids2, from_training=True)  
#         X_train2, y_train2, _, _ = self._get_data_from_indices(tmp_train_ids2, from_training=False)    
#         mod = mod.fit(X_train2, y_train2)
            
#         X_val, y_val, meta_dat, _ = self._get_data_from_indices(val_ids) 
        X_val, y_val, meta_dat, _ = self._get_data_from_indices(tmp_val_ids, from_where=f'_VAL_ts_{ts_serial}_tr_{tr_serial}_mod_{mod_serial}')   
#         X_val, y_val, meta_dat, _ = self._get_data_from_indices(tmp_train_ids)   
        y_pred = mod.predict(X_val)  
        y_pred_proba = mod.predict_proba(X_val) 
        target_and_prediction_df.reset_index(drop=True, inplace=True) 
        meta_dat.reset_index(drop=True, inplace=True) 
        target_and_prediction_df = pd.concat([target_and_prediction_df, meta_dat]) 
        target_and_prediction_df[self.class_name] = y_val 
        # target_and_prediction_df[f"Prediction_{str(model.__class__.__name__)}"] = y_pred 
        target_and_prediction_df[f"Prediction_{ML_Classifiers.get_short_form(str(classifier_method.value))}"] = y_pred  
        for p_ind in range(y_pred_proba.shape[1]):
            target_and_prediction_df[f"Prediction_Proba_{p_ind}_{ML_Classifiers.get_short_form(str(classifier_method.value))}"] = y_pred_proba[:, p_ind].tolist()
            
        model.feature_names = self.selected_features 
        model.feature_importance_scores = importance_scores
        model_scores = self.calculate_model_scores(model, y_val, y_pred, y_pred_proba) 

        self.logger.info(f"""
        Best model (GriveSearchCV): {model} 
        Best model: {mod} 
        Best estimator of the model: {mod_est} 
        Best parameters of the model: {mod_par} 
        Best model scores: {model_scores} 
        ---------------------------------------------------------------------------------------------------
        """)

        return model, model_scores, target_and_prediction_df, fold_info_df 
        # return model, model_scores, target_and_prediction_df, fold_info_df


    def call_all_model_optimization(self, classifier_method, parameters, train_ids, val_ids, ts_serial, tr_serial, mod_serial, parameter_optimization):
        model, model_scores, target_and_prediction, fold_info_df = None, None, None, None 
        if parameter_optimization == 1:
            model, model_scores, target_and_prediction, fold_info_df = self.run_model_gridSearch(classifier_method, parameters, train_ids, val_ids, ts_serial, tr_serial, mod_serial)
        elif parameter_optimization == 2:
            model, model_scores, target_and_prediction, fold_info_df = self.run_model_randomizedSearch(classifier_method, parameters, train_ids, val_ids, ts_serial, tr_serial, mod_serial)
        elif parameter_optimization == 3:
            model, model_scores, target_and_prediction, fold_info_df = self.run_model_baysianSearch(classifier_method, parameters, train_ids, val_ids, ts_serial, tr_serial, mod_serial)
        elif parameter_optimization == 4:
            model, model_scores, target_and_prediction, fold_info_df = self.run_model_customGridSearch(classifier_method, parameters, train_ids, val_ids, ts_serial, tr_serial, mod_serial)
        return model, model_scores, target_and_prediction, fold_info_df


    def get_ml_model_instances(self, classifier_method, parameters=None):
        classifier = None

        ### GPU code START
        global GPUs
        global HAS_GPU

        # GPUs = GPUtil.getGPUs()
        # tot_gpus = len(GPUs)
        # HAS_GPU = True if len(GPUs) > 0 else False
        # avl_GPUIDs = GPUtil.getAvailable(order = 'first', limit = tot_gpus, maxLoad = 0.5, maxMemory = 0.5, includeNan=False, excludeID=[], excludeUUID=[])
        # tot_avl_gpus = len(avl_GPUIDs)
        # print(f'For GPU based tasks. There are {tot_gpus} GPUs in the system and {tot_avl_gpus} are available. \nAvailable GPU IDs are: {avl_GPUIDs}')
        allGPUs, bestGPU = HumachLab_Global.get_gpu_details(show_logs=False)
        ### GPU code END

        # ####### rf #######
        # rf - random_forest classifier
        if classifier_method == ML_Classifiers.RF:
            classifier = RandomForestClassifier() if (parameters is None) else RandomForestClassifier(parameters)
        # ####### knn #######
        # knn - k_neares_neighbours classifier
        elif classifier_method == ML_Classifiers.kNN:
            classifier = KNeighborsClassifier() if (parameters is None) else KNeighborsClassifier(parameters)
        # ####### nb #######
        # knn - naieve bayes classifier
        elif classifier_method == ML_Classifiers.NB:
            classifier = GaussianNB() if (parameters is None) else GaussianNB(parameters)
        # ####### svm/svc #######
        # knn - support vector classifier
        elif classifier_method == ML_Classifiers.SVC:
            classifier = SVC() if (parameters is None) else SVC(parameters)
        # ####### knn #######
        # knn - k_neares_neighbours classifier
        elif classifier_method == ML_Classifiers.DT:
            classifier = DecisionTreeClassifier() if (parameters is None) else DecisionTreeClassifier(parameters)
        # ####### LogReg #######
        # LogReg - logistic regression classifier
        elif classifier_method == ML_Classifiers.LogReg:
            classifier = LogisticRegression() if (parameters is None) else LogisticRegression(parameters)
        # ####### GBoost #######
        # GBoost - gradient boosting classifier
        elif classifier_method == ML_Classifiers.GBoost:
            classifier = GradientBoostingClassifier() if (parameters is None) else GradientBoostingClassifier(parameters)
        # ####### XGBoost #######
        # GBoost - eXtreme gradient boosting classifier
        elif classifier_method == ML_Classifiers.XGBoost:
            classifier = XGBClassifier() if (parameters is None) else XGBClassifier(parameters)

        ### GPU code - Comment it if no gpu available or not linux system or no support for RapidsAI package
        # ####### gpu-rf #######
        # gpu-rf - gpu-random_forest classifier
        # elif classifier_method == ML_Classifiers.GPURF and tot_avl_gpus>0:
        #     classifier = gpuRandomForestClassifier() if (parameters is None) else gpuRandomForestClassifier(parameters)

        # ####### None #######
        # No classifier
        else:
            self.logger.info(f'No classifier is selected...')

        # ####### ####### #######
        return classifier


    def get_ml_scoring_metrices(self, reft=None):
        model_scoring_mets = [ML_Performace_Metrics.ACC, ML_Performace_Metrics.PREC, ML_Performace_Metrics.RECL,
                              ML_Performace_Metrics.SEN, ML_Performace_Metrics.SPEC, ML_Performace_Metrics.FPR,
                              ML_Performace_Metrics.FNR, ML_Performace_Metrics.F1, ML_Performace_Metrics.ROC_AUC]

        scoring = [ML_Performace_Metrics.ACC.value]
        bst_mod_mets_1 = None
        i = 0
        for met in self.best_model_scoring_metrics:
            if i==0:
                scoring.clear()
                if (reft is not None):
                    if reft == ML_Performace_Metrics.F1SCR:
                        reft = ML_Performace_Metrics.F1
                    if (reft not in model_scoring_mets):
                        reft = None

            if met == ML_Performace_Metrics.F1SCR:
                met = ML_Performace_Metrics.F1

            if met in model_scoring_mets:
                scoring.append(met.value)
            i += 1

        refit = (scoring[0]) if reft is None else reft.value
        
#         scoring = [ML_Performace_Metrics.F1]
#         refit = True

        return scoring, refit


    ############################################################################
    def get_parameters_for_ml_models(self, classifier_method):
        parameters = {}
        if not self.should_use_params:
            return parameters

        # Parameter generation method name
        method_name = f'{str(classifier_method.value)}_parameters'

        try:
            method = getattr(self, method_name)
            # Call method for parameter generation
            self.logger.info(f'Calling method: {method_name}')
            parameters = method()
        except AttributeError:
            self.logger.warning(f'No such method exists with the name: {method_name}')
            raise NotImplementedError("Class `{}` does not implement `{}`".format(self.__class__.__name__, method_name))

        # ####### ####### #######
        return parameters




    ############################################################################
    def generate_parameter_dictionary(self, par_names, par_vals, par_ind):
        self.logger.info(f'All parameters: {par_names}, {par_vals}, {par_ind}')
        final_par_names = []
        par_dict = {}

#         for i in par_ind:
#             pn = par_names[i]
#             pv = par_vals[i]
#             exec(f'{pn}={pv}')
#             final_par_names.append(pn)
        
        sel_par = [pp for ii,pp in enumerate(par_names) if ii in par_ind] 
        for (pn, pv) in zip(sel_par, par_vals):
            exec(f'{pn}={pv}')
            final_par_names.append(pn)

        for par in final_par_names:
            par_dict[par] = eval(par)

        return par_dict


    # def float_range(self, start, stop, step):
    #     start = decimal.Decimal(start)
    #     stop = decimal.Decimal(stop)
    #     while start < stop:
    #         yield float(start)
    #         start *= decimal.Decimal(step)


    # #########################################################################
    # Model parameter settings
    # #########################################################################
    # ### ML Classifier Method Parameters
    
    
    
    def logistic_regression_parameters(self):

        # ### Parameter generation using function
        par_names = ['penalty', 'solver', 'max_iter', 'C']
        par_vals = [['l1', 'l2', 'elasticnet'],
                    ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
                    list(range(50, 5000, 10)),
                    list(HumachLab_StaticMethods.float_range('0.001', '1.0', '0.01'))]

        par_vals = [[30, 50, 75, 100, 200, 500, 750, 1000], [2, 3, 5, 7], [5, 7, 11, 15, 21, 30, 50, 75, 100, 200, 500, 750, 1000]]
        par_vals = [[5, 7, 11, 15, 21, 30, 50, 75, 100, 200, 500, 750, 1000]]
        par_vals = [[15, 21, 30, 50, 75, 100, 200, 500]]
        par_vals = [[50, 75, 100]]
        par_vals = [[15, 21, 30, 50, 75, 100, 200, 500], [3, 5, 7, 10, 15, 20, 25, 30]]
        par_vals = [['l1', 'l2', 'elasticnet'],[50, 100, 130, 150, 170, 200, 250, 350, 500, 750, 1000]]
        par_vals = [['l1', 'l2', 'elasticnet'],[50, 100, 130, 150, 170, 200],[0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0]]
        par_vals = [['l2', 'elasticnet'],[50, 100, 130, 150, 170, 200],[0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0]]
        par_ind = [0, 2, 3]
        parameters = self.generate_parameter_dictionary(par_names, par_vals, par_ind)

        return parameters
    
    
    
    def k_nearest_neighbors_parameters(self):

        # ### Parameter generation using function
        par_names = ['n_neighbors', 'p', 'metric', 'n_splits']
        par_vals = [list(range(2, 100)),
                    list(range(2, 100)),
                    ['manhattan', 'minkowski', 'euclidean'],
                    list(range(2, 10))]

        par_vals = [list(range(100, 1000, 50)), list(range(2, 11, 1)), [2, 3, 5, 9, 13, 19, 29]]
        par_vals = [[2, 3, 5, 9, 13, 19, 29]]
        par_vals = [[2, 3, 5, 10, 15, 25, 35], ['manhattan', 'minkowski', 'euclidean']]
        par_ind = [0, 2]
        parameters = self.generate_parameter_dictionary(par_names, par_vals, par_ind)

        return parameters
    
    
    
    def naive_bayes_parameters(self):

        # ### Parameter generation using function
        par_names = ['var_smoothing']
        par_vals = [list(np.logspace(0, -9, num=100))]
        par_vals = [list(np.logspace(0, -9, num=100))]

        # par_vals = []
        # par_vals = []
        par_vals = [list(np.logspace(0,-9, num=5))]
        par_ind = [0]
        parameters = self.generate_parameter_dictionary(par_names, par_vals, par_ind)

        return parameters


    def support_vector_classifier_parameters(self):

        # ### Parameter generation using function #, probability=True
        par_names = ['probability', 'C', 'kernel', 'gamma', 'degree', 'class_weightdict']
        par_vals = [[True],
                    list(HumachLab_StaticMethods.float_range('0.001', '1', '0.01')),
                    ['linear', 'rbf', 'poly', 'sigmoid'],
                    list(HumachLab_StaticMethods.float_range('0.000001', '1', '10')),
                    list(range(1, 10)),
                    [None, 'balanced']]

        # par_vals = [list(HumachLab_StaticMethods.float_range('0.000001', '1', '10')), list(HumachLab_StaticMethods.float_range('0.00001', '1', '10')), list(HumachLab_StaticMethods.float_range('0.0001', '1', '10'))]
        par_vals = [list(HumachLab_StaticMethods.float_range('0.001', '1.', '0.1')), ['linear', 'rbf', 'poly']]
        par_vals = [[True],[0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0], ['linear', 'rbf', 'poly']]
        par_ind = [0, 1, 2]
        parameters = self.generate_parameter_dictionary(par_names, par_vals, par_ind)

        return parameters
    
    
    
    def decision_tree_parameters(self):

        # ### Parameter generation using function
        par_names = ['max_depth', 'criterion', 'splitter', 'min_samples_split', 'min_samples_leaf', 'max_leaf_nodes']
        par_vals = [list(range(1, 100)),
                    ['gini', 'entropy', 'log_loss'],
                    ['best', 'random'],
                    list(range(1, 10)),
                    list(range(1, 10)),
                    list(range(1, 100))]

        par_vals = [list(range(1, 100)), list(range(1, 100, 2)), list(range(1, 100, 3))]
        par_vals = [list(range(1, 100))]
        par_vals = [[2, 3, 5, 7, 10, 15, 20, 25, 30], ['gini', 'entropy', 'log_loss']]
        par_ind = [0, 1]
        parameters = self.generate_parameter_dictionary(par_names, par_vals, par_ind)

        return parameters
    
    
    
    def random_forest_parameters(self): 

        # ### Parameter generation using function
        par_names = ['n_estimators', 'max_depth', 'criterion', 'splitter', 'max_features', 'min_samples_split', 'min_samples_leaf', 'max_leaf_nodes']
        par_vals = [list(range(1, 500, 5)),
                    list(range(1, 100)),
                    ['gini', 'entropy', 'log_loss'], 
                    ['best', 'random'],
                    list(range(2, 20, 1)),
                    list(range(1, 10)),
                    list(range(1, 10)),
                    list(range(1, 100))]

        par_vals = [[30, 50, 75, 100, 200, 500, 750, 1000], [2, 3, 5, 7], [5, 7, 11, 15, 21, 30, 50, 75, 100, 200, 500, 750, 1000]]
        par_vals = [[5, 7, 11, 15, 21, 30, 50, 75, 100, 200, 500, 750, 1000]]
        par_vals = [[15, 21, 30, 50, 75, 100, 200, 500]]
        par_vals = [[50, 75, 100]]
        par_vals = [[3, 5, 10, 15, 21, 30, 50, 75, 100], [2, 3, 5, 7, 10, 15, 20, 25, 30], ['gini', 'entropy', 'log_loss']]
        # par_vals = [[2, 3, 5, 7]]
        par_ind = [0, 1, 2]
        parameters = self.generate_parameter_dictionary(par_names, par_vals, par_ind)

        return parameters
    
    
    
    def graphics_processing_unit_random_forest(self):

        # ### Parameter generation using function
        par_names = ['n_estimators', 'n_bins', 'n_streams', 'max_depth', 'max_features', 'criterion', 'splitter', 'min_samples_split', 'min_samples_leaf', 'max_leaf_nodes']
        par_vals = [list(range(1, 500, 5)),
                    list(range(1, 100)),
                    list(range(2, 20, 1)),
                    ['gini', 'entropy', 'log_loss'], 
                    ['best', 'random'],
                    list(range(1, 10)),
                    list(range(1, 10)),
                    list(range(1, 100))]

        par_vals = [[30, 50, 75, 100, 200, 500, 750, 1000], [2, 3, 5, 7], [5, 7, 11, 15, 21, 30, 50, 75, 100, 200, 500, 750, 1000]]
        par_vals = [[5, 7, 11, 15, 21, 30, 50, 75, 100, 200, 500, 750, 1000], 15, 8]
        # par_vals = [[2, 3, 5, 7]]
        par_ind = [0, 1, 2]
        parameters = self.generate_parameter_dictionary(par_names, par_vals, par_ind)

        return parameters
    
    
    
    def gradient_boosting_parameters(self):

        # ### Parameter generation using function 
        par_names = ['n_estimators', 'max_depth', 'learning_rate', 'max_features', 'loss', 'min_samples_split', 'min_samples_leaf']
        par_vals = [list(range(1, 500, 5)),
                    list(range(1, 100)),
                    list(HumachLab_StaticMethods.float_range('0.001', '1', '0.01')), 
                    list(range(2, 20, 1)),
                    ['log_loss', 'exponential'],
                    list(range(1, 10)),
                    list(range(1, 10))]

        par_vals = [[30, 50, 75, 100, 200, 500, 750, 1000], [2, 3, 5, 7], [5, 7, 11, 15, 21, 30, 50, 75, 100, 200, 500, 750, 1000]]
        par_vals = [[5, 7, 11, 15, 21, 30, 50, 75, 100, 200, 500, 750, 1000]]
        par_vals = [[15, 21, 30, 50, 75, 100, 200, 500]]
        par_vals = [[50, 75, 100]]
        par_vals = [[15, 21, 30, 50, 75, 100, 200, 500], [3, 5, 7, 10, 15, 20, 25, 30], [0.01, 0.05, 0.1, 0.5, 1.0]]
        par_vals = [[3, 5, 10, 15, 21, 30, 50, 75, 100], [2, 3, 5, 7, 10, 15, 20, 25, 30], [0.01, 0.05, 0.1, 0.5, 1.0]]
        # par_vals = [[2, 3, 5, 7]]
        par_ind = [0, 1, 2]
        parameters = self.generate_parameter_dictionary(par_names, par_vals, par_ind)

        return parameters
    
    
    
    def xtreme_gradient_boosting_parameters(self): #xtreme_gradient_boosting

        # ### Parameter generation using function 
        par_names = ['max_depth', 'eta', 'max_leaves']
        par_vals = [list(range(1, 100)),
                    list(HumachLab_StaticMethods.float_range('0.001', '1', '0.01')),
                    list(range(0, 20, 1)) ]

        par_vals = [[3, 6, 10, 15, 25, 40, 100, 250, 500, 750, 1000], [0.01, 0.05, 0.1, 0.3, 0.5, 1.0], [0, 5, 7, 11, 15, 21, 30, 50, 75, 100, 200, 500, 750, 1000]]
        par_vals = [[2, 3, 6, 10, 15, 20, 25, 30], [0.01, 0.05, 0.1, 0.3, 0.5, 1.0]]
        # par_vals = [[2, 3, 5, 7]]
        par_ind = [0, 1]
        parameters = self.generate_parameter_dictionary(par_names, par_vals, par_ind)

        return parameters
    
    
    


    # #########################################################################
    # Calculate and save classification details and model scores
    # #########################################################################
    #############################

    def calculate_model_scores(self, mods, y_test, y_pred, y_pred_proba): 
#         print(y_test, '\n', y_pred, '\n', y_pred_proba, '\n')
#         target_labels = np.unique(np.array(y_test)).tolist() 
        target_labels = sorted( self.dataset[self.class_name].unique().tolist() )
        
        y_pred = y_pred.tolist() 
        perf_scores = self.calculate_performance_scores(y_test, y_pred, y_pred_proba, labels=target_labels)  # average = 'weighted', 'macro', 'micro' 
        confMat = perf_scores['Conf_Mat']

        acc = round(perf_scores['ACC'], 3) #Accuracy score or total correct prediction rate 
        prec = round(perf_scores['PREC'], 3) #precision or positive predictive value (PPV)
        reca_sens = round(perf_scores['REC'], 3) #sensitivity, recall, hit rate, or true positive rate (TPR)
        spec = round(perf_scores['SPE'], 3) #specificity, selectivity or true negative rate (TNR)
        f1sc = round(perf_scores['F1SCR'], 3)
        auc_s = round(perf_scores['AUC'], 3) 
        
        scr_dict = {'method': str(mods), 'model': mods.best_estimator_, 'model_parameters': mods.best_params_, 
                    'model_scores': round(mods.best_score_*100,2),
                    ML_Performace_Metrics.CONF_MAT.value: confMat, ML_Performace_Metrics.ACC.value: acc, ML_Performace_Metrics.PREC.value: prec,
                    ML_Performace_Metrics.RECL.value: reca_sens, ML_Performace_Metrics.SEN.value: reca_sens, ML_Performace_Metrics.SPEC.value: spec,
                    ML_Performace_Metrics.F1SCR.value: f1sc, ML_Performace_Metrics.ROC_AUC.value: auc_s}
        
#         scr_dict = {'method_class': str(mods.__class__.__name__), 'model_name': mods.best_estimator_, 'model_parameters': mods.best_params_, 
#                     'model_scores': round(mods.best_score_*100,2),
#                     ML_Performace_Metrics.CONF_MAT.value: confMat, ML_Performace_Metrics.ACC.value: acc, ML_Performace_Metrics.PREC.value: prec,
#                     ML_Performace_Metrics.RECL.value: reca_sens, ML_Performace_Metrics.SEN.value: reca_sens, ML_Performace_Metrics.SPEC.value: spec,
#                     ML_Performace_Metrics.F1SCR.value: f1sc, ML_Performace_Metrics.ROC_AUC.value: auc_s}
#         scr_dict = {'model_class': str(mods.__class__.__name__), 'method_name': mods.estimator, 'method_parameters': mods.best_params_, 
#                     'method_scores': round(mods.best_score_*100,2),
#                     ML_Performace_Metrics.CONF_MAT.value: confMat, ML_Performace_Metrics.ACC.value: acc, ML_Performace_Metrics.PREC.value: prec,
#                     ML_Performace_Metrics.RECL.value: reca_sens, ML_Performace_Metrics.SEN.value: reca_sens, ML_Performace_Metrics.SPEC.value: spec,
#                     ML_Performace_Metrics.FPR.value: fpr, ML_Performace_Metrics.FNR.value: fnr, ML_Performace_Metrics.F1SCR.value: f1sc, ML_Performace_Metrics.ROC_AUC.value: auc_s}
#         scr_dict = {'model_class': str(mods.__class__.__name__), 'method_name': str(mods), 'method_parameters': mods.best_params_, 
#                     'method_scores': round(mods.best_score_*100,2),
#                     ML_Performace_Metrics.CONF_MAT.value: confMat, ML_Performace_Metrics.ACC.value: acc, ML_Performace_Metrics.PREC.value: prec,
#                     ML_Performace_Metrics.RECL.value: reca_sens, ML_Performace_Metrics.SEN.value: reca_sens, ML_Performace_Metrics.SPEC.value: spec,
#                     ML_Performace_Metrics.FPR.value: fpr, ML_Performace_Metrics.FNR.value: fnr, ML_Performace_Metrics.F1SCR.value: f1sc, ML_Performace_Metrics.ROC_AUC.value: auc_s}
        scr_df = pd.DataFrame([list( scr_dict.values() )], columns=list( scr_dict.keys() )) 
        self.logger.info(f"""Score columns: {scr_df.shape} {scr_df.columns.values.tolist()}""") 

        return scr_df
    
    
    def calculate_performance_scores(self, y_true, y_pred, y_pred_proba, labels=[0, 1], verbose=2, average='weighted'): # average = 'macro', 'micro', 'weighted' 
        #### SOURCES: https://www.youtube.com/watch?v=PCHf_7jBor8 
        # https://www.mariakhalusova.com/posts/2019-04-17-ml-model-evaluation-metrics-p2/ 
        # https://www.mariakhalusova.com/posts/2019-04-11-ml-model-evaluation-metrics-p1/
        # https://www.evidentlyai.com/classification-metrics/multi-class-metrics 
        # https://www.kaggle.com/code/nkitgupta/evaluation-metrics-for-multi-class-classification 

        model_scores = []
        true_label_uniq = np.unique(np.array(y_true)).tolist() 
        print(np.unique(np.array(y_true)), np.unique(np.array(y_pred)))
        print(y_true, y_pred) 
        conf_matrix = confusion_matrix(y_true, y_pred, labels=labels).tolist()
        print(np.array(conf_matrix) )

        ### For micro averaging and binary class 
        conf_matrix_arr = np.array(conf_matrix) 
        one_vs_all_confMat = []     
        for label in labels:
            tp_lbl = conf_matrix_arr[label, label] 
            fp_lbl = np.sum(conf_matrix_arr[:, label])-tp_lbl 
            fn_lbl = np.sum(conf_matrix_arr[label, :])-tp_lbl 
            tn_lbl = np.sum(conf_matrix_arr)-(tp_lbl+fp_lbl+fn_lbl) 
            one_vs_all_confMat.append([tn_lbl, fp_lbl, fn_lbl, tp_lbl]) 
        print(np.array(one_vs_all_confMat)) 

        tn_tot = np.sum( np.array(one_vs_all_confMat)[:, 0] ) 
        fp_tot = np.sum( np.array(one_vs_all_confMat)[:, 1] )  
        fn_tot = np.sum( np.array(one_vs_all_confMat)[:, 2] )  
        tp_tot = np.sum( np.array(one_vs_all_confMat)[:, 3] )

        conf_matrix_tol = [[tn_tot, fp_tot], [fn_tot, tp_tot]] 
        print(np.array(conf_matrix_tol)) 

        if len(labels)==2:
            tn_tot = one_vs_all_confMat[1][0] 
            fp_tot = one_vs_all_confMat[1][1] 
            fn_tot = one_vs_all_confMat[1][2] 
            tp_tot = one_vs_all_confMat[1][3] 
            average = "micro"

        result = [] 
        for label in labels:
            precision, recall, fscore, support = precision_recall_fscore_support( np.array(y_true)==label, np.array(y_pred)==label ) 
            # tmp_fpr, tmp_tpr, tmp_thresholds = roc_curve(np.array(y_true)==label, np.array(y_pred)==label, pos_label=label) 
            tmp_fpr, tmp_tpr, tmp_thresholds = roc_curve(np.array(y_true)==label, np.array(y_pred)==label) 
            auc_score = auc(tmp_fpr, tmp_tpr)*100 

            if label in true_label_uniq: 
                result.append( [label, precision[1], recall[1], recall[1], recall[0], fscore[1], auc_score, support[1]] ) 
            else:
                result.append( [label, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0] ) 

            accuracy = accuracy_score(np.array(y_true)==label, np.array(y_pred)==label)*100 
            if verbose>1:
                print(
                    f'Class-wise info: For multilevel internal scores fo label {label}: \n', 
                    f'Accuracy = {accuracy}\n', 
                    f'Precision = {precision}\n', 
                    f'Recall = {recall}\n', 
                    f'F1 score = {fscore}\n', 
                    f'AUC score = {auc_score}\n', 
                    f'Support = {support}\n', 
                )
        tdf = pd.DataFrame(result, columns=['Label', 'Precision', 'Recall', 'Sensitivity', 'Specificity', 'F1 Score', 'AUC', 'Support']) 

        if average=='macro': #average = "weighted", "macro", "micro" 
            tdf = tdf[['Precision', 'Recall', 'Sensitivity', 'Specificity', 'F1 Score', 'AUC']].apply(lambda col:np.mean(col), axis=0) 
        elif average=='micro':
            prc = (tp_tot / (tp_tot+fp_tot))*100 if (tp_tot+fp_tot)!=0 else 0.0 #precision or positive predictive value (PPV)
            rec = (tp_tot / (tp_tot+fn_tot))*100 if (tp_tot+fn_tot)!=0 else 0.0 #sensitivity, recall, hit rate, or true positive rate (TPR)
            sns = rec #sensitivity same as recall 
            spc = (tn_tot / (tn_tot+fp_tot))*100 if (tn_tot+fp_tot)!=0 else 0.0 #specificity, selectivity or true negative rate (TNR)
            f1s = (2*tp_tot / (2*tp_tot+fp_tot+fn_tot))*100 if (2*tp_tot+fp_tot+fn_tot)!=0 else 0.0 #specificity, selectivity or true negative rate (TNR)  
            auc_s = roc_auc_score(y_true, y_pred) if len(labels)==2 else roc_auc_score(y_true, y_pred_proba, multi_class='ovr', average=average) #multi_class='ovo', 'ovr' 
#             auc_s = 0.5
#             if len(labels)==2:
#                 auc_s = roc_auc_score(y_true, y_pred)  
#             else:
#                 auc_s = roc_auc_score(y_true, y_pred_proba, multi_class='ovr', average=average) #multi_class='ovo', 'ovr' 
            tdf = pd.Series([prc, rec, sns, spc, f1s, auc_s], index=['Precision', 'Recall', 'Sensitivity', 'Specificity', 'F1 Score', 'AUC'])  
        else: ## Default = weighted
            class_weights = tdf['Support']/tdf['Support'].sum() 
            tdf = tdf[['Precision', 'Recall', 'Sensitivity', 'Specificity', 'F1 Score', 'AUC']].apply(lambda col:np.sum(col*class_weights), axis=0) 

        acc = accuracy_score(y_true, y_pred)*100 
        tdf_summary = pd.Series([conf_matrix, acc, tdf['Precision'], tdf['Recall'], tdf['Sensitivity'], tdf['Specificity'], tdf['F1 Score'], tdf['AUC']],
                               index=['Conf_Mat', 'ACC', 'PREC', 'REC', 'SEN', 'SPE', 'F1SCR', 'AUC'])

        if verbose>1:
            confMat = tdf_summary['Conf_Mat']
            acc = round(tdf_summary['ACC'], 3) #Accuracy score or total correct prediction rate 
            prec = round(tdf_summary['PREC'], 3) #precision or positive predictive value (PPV)
            reca_sens = round(tdf_summary['REC'], 3) #sensitivity, recall, hit rate, or true positive rate (TPR)
            spec = round(tdf_summary['SPE'], 3) #specificity, selectivity or true negative rate (TNR)
            f1sc = round(tdf_summary['F1SCR'], 3)
            auc_s = round(tdf_summary['AUC'], 3) 
            print(
                f'CLASSIFICATION MERICS:\n',
                f'{"_"*55}\n',
                f'Confusion Matrix: \n{np.array(conf_matrix)}\n',
                f'Accuracy (acc): {acc}\n',
                f'Precision (prc): {prec}\n',
                f'Recall (rec): {reca_sens}\n',
                f'Sensitivity (sns): {reca_sens}\n',
                f'Specificity (spc): {spec}\n',
                f'F1 Score (f1s): {f1sc}\n',
                f'ROC AUC (AUC): {auc_s}',
            )

        return tdf_summary

